In [2]:
import matplotlib
from matplotlib import font_manager

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv

fontpaths = ["C:/Users/kwon3/AppData/Local/Microsoft/Windows/Fonts/NanumGothicBold.ttf"]
font_files = font_manager.findSystemFonts(fontpaths=fontpaths)

for ff in font_files:
    font_manager.fontManager.addfont(ff)

matplotlib.rc('font', family="Malgun Gothic")
matplotlib.rcParams['axes.unicode_minus'] = False

'sudo'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'sudo'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

- 데이터 경로 변수

In [4]:
# DATA_PATH = "/content/drive/MyDrive/파이널프로젝트/data/"
# DATA_PATH

In [5]:
DATA_PATH = "C:/practice/finalproject/data/"
DATA_PATH

'C:/practice/finalproject/data/'

- 시드값

In [6]:
SEED = 42

- 데이터 불러오기

In [7]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

- 공통 피처 파일 불러오기

In [8]:
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

In [9]:
#train_ft = pd.read_csv(f"{DATA_PATH}train_common_2.csv") # 학습 데이터(피처)
#test_ft = pd.read_csv(f"{DATA_PATH}test_common_2.csv") # 테스트 데이터(피처)

#train_ft.shape , test_ft.shape

((14940, 2331), (12225, 2331))

In [10]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_1114.csv") # 학습 데이터(피처)
test_ft = pd.read_csv(f"{DATA_PATH}test_common_1114.csv") # 테스트 데이터(피처)

train_ft.shape , test_ft.shape

((14940, 2331), (12225, 2331))

# 결측치 처리

In [11]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

Series([], dtype: int64)

In [12]:
mask = test_ft.isnull().sum() > 0
test_ft.isnull().sum()[mask]

Series([], dtype: int64)

# 특성 공학(Feature Engineering)

- ID 변수 제외

In [13]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 2330), (12225, 2330))

- 추가 피처 만들어 보기

In [14]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_지점코드_") ]

train_ft["지점코드별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["지점코드별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["지점코드별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["지점코드별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["지점코드별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["지점코드별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2333), (12225, 2333))

In [15]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_중분류_") ]  #중분류는 그대로쓰고, 다른건 차원축소하지많고,,, 

train_ft["중분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["중분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["중분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["중분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["중분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["중분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2336), (12225, 2336))

In [16]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_대분류_") ]

train_ft["대분류별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["대분류별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["대분류별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["대분류별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["대분류별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["대분류별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2339), (12225, 2339))

In [17]:
cols = [ col for col in train_ft.columns if col.startswith("pivot_브랜드코드_") ] #써야하면 차원축소를 할것(버리던가,, 차원축소해서 옆으로붙이기) 

train_ft["브랜드코드별_구매횟수_std"] = train_ft[cols].std(axis=1)
train_ft["브랜드코드별_구매횟수_skew"] = train_ft[cols].skew(axis=1)
train_ft["브랜드코드별_구매횟수_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["브랜드코드별_구매횟수_std"] = test_ft[cols].std(axis=1)
test_ft["브랜드코드별_구매횟수_skew"] = test_ft[cols].skew(axis=1)
test_ft["브랜드코드별_구매횟수_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2342), (12225, 2342))

In [18]:
cols = [ col for col in train_ft.columns if col.endswith("요일_구매비율") ]

train_ft["요일별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["요일별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["요일별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["요일별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["요일별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["요일별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2345), (12225, 2345))

In [19]:
cols = [ col for col in train_ft.columns if col.endswith("월_구매비율") ]

train_ft["월별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["월별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["월별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["월별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["월별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["월별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2348), (12225, 2348))

In [20]:
cols = ["봄_구매비율","여름_구매비율","가을_구매비율","겨울_구매비율"]

train_ft["계절별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["계절별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["계절별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["계절별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["계절별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["계절별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2351), (12225, 2351))

In [21]:
cols = [ col for col in train_ft.columns if col.endswith("분기_구매비율") ]

train_ft["분기별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["분기별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["분기별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["분기별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["분기별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["분기별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2354), (12225, 2354))

In [22]:
cols = [ col for col in train_ft.columns if col.endswith("년_구매비율") ]

train_ft["년도별_구매비율_std"] = train_ft[cols].std(axis=1)
test_ft["년도별_구매비율_std"] = test_ft[cols].std(axis=1)

train_ft.shape, test_ft.shape

((14940, 2355), (12225, 2355))

In [23]:
cols = ["봄_구매비율","여름_구매비율","가을_구매비율","겨울_구매비율"]

train_ft["계절별_구매비율_std"] = train_ft[cols].std(axis=1)
train_ft["계절별_구매비율_skew"] = train_ft[cols].skew(axis=1)
train_ft["계절별_구매비율_kurt"] = train_ft[cols].kurt(axis=1)

test_ft["계절별_구매비율_std"] = test_ft[cols].std(axis=1)
test_ft["계절별_구매비율_skew"] = test_ft[cols].skew(axis=1)
test_ft["계절별_구매비율_kurt"] = test_ft[cols].kurt(axis=1)

train_ft.shape, test_ft.shape

((14940, 2355), (12225, 2355))

In [24]:
cols = [ col for col in train_ft.columns if col.startswith("18시") ]

train_ft["18시기준_구매비율_std"] = train_ft[cols].std(axis=1)

test_ft["18시기준_구매비율_std"] = test_ft[cols].std(axis=1)

train_ft.shape, test_ft.shape

((14940, 2356), (12225, 2356))

In [25]:
# 주말에 중분류_아동용품_구매하는 비율(가족단위의 아이들용품 구매가 많지 않으까?)

train_ft["주말_중_아동용품_구매비율"] = train_ft["주말_구매비율"] * train_ft["주구매_중분류_아동용품"]

test_ft["주말_중_아동용품_구매비율"] = test_ft["주말_구매비율"] * test_ft["주구매_중분류_아동용품"]

train_ft.shape, test_ft.shape

((14940, 2357), (12225, 2357))

In [26]:
# 주말에 중분류_아동용품_구매하는 비율(가족단위의 아이들용품 구매가 많지 않으까?)

train_ft["주말_대_아동용품_구매비율"] = train_ft["주말_구매비율"] * train_ft["주구매_대분류_아동용품"]

test_ft["주말_대_아동용품_구매비율"] = test_ft["주말_구매비율"] * test_ft["주구매_대분류_아동용품"]

train_ft.shape, test_ft.shape

((14940, 2358), (12225, 2358))

In [27]:
# 18시이전이면서 주말에_중분류_아동용품_구매하는 비율
train_ft["18시이후_주말_중_아동용품_구매비율"] = train_ft["18시이후_구매비율"] * train_ft["주말_중_아동용품_구매비율"]
test_ft["18시이후_주말_중_아동용품_구매비율"] = test_ft["18시이후_구매비율"] * test_ft["주말_중_아동용품_구매비율"]

train_ft.shape, test_ft.shape

((14940, 2359), (12225, 2359))

In [28]:
train_ft.head()

총방문일수       첫구매날짜     마지막구매날짜  백화점이용기간  구매주기  주말방문일수  일별평균구매건수  구매간격_표준편차  \
0      9  2004-05-07  2005-01-16      255    28       2  1.555556  22.744789   
1     21  2004-05-11  2005-04-27      352    16       1  2.000000  11.739134   
2     55  2004-05-06  2005-04-26      356     6      11  1.854545   5.019987   
3     90  2004-05-11  2005-04-20      345     3      19  2.055556   3.137146   
4     24  2004-05-09  2005-03-17      313    13       6  1.500000  11.962731   

     평균구매요일  주구매요일  ...  계절별_구매비율_skew  계절별_구매비율_kurt  분기별_구매비율_std  \
0  3.000000      2  ...      -1.539601       2.888889      0.148690   
1  2.309524      3  ...      -0.123691      -5.290173      0.090141   
2  2.333333      0  ...       1.295669       2.179478      0.127702   
3  3.081081      4  ...       1.902132       3.642104      0.073505   
4  3.861111      4  ...       0.729678      -1.947285      0.188398   

   분기별_구매비율_skew  분기별_구매비율_kurt  년도별_구매비율_std  18시기준_구매비율_std  주말_중_아동용품_구매비율  \
0       0.000000       0.390533      0.606092        0.202031        0.030612   
1       1.658524       2.615468      0.370389        0.269374        0.000000   
2       0.238913      -4.582219      0.013865        0.499134        0.000000   
3       0.281261      -0.460929      0.080266        0.393686        0.002893   
4       1.794946       3.390359      0.471405        0.353553        0.010031   

   주말_대_아동용품_구매비율  18시이후_주말_중_아동용품_구매비율  
0        0.030612              0.010933  
1        0.000567              0.000000  
2        0.032295              0.000000  
3        0.051103              0.000641  
4        0.040123              0.002508  

[5 rows x 2359 columns]

In [29]:
mask = train_ft.isnull().sum() > 0
train_ft.isnull().sum()[mask]

Series([], dtype: int64)

In [30]:
#다이아몬드 혹은 플래티넘 등급의 아동용품 구매비율? ->의미가 없을것같긴한데,,, 




## Feature Encoding

In [31]:
train_ft["첫구매날짜"] = pd.to_datetime(train_ft["첫구매날짜"])
train_ft["마지막구매날짜"] = pd.to_datetime(train_ft["마지막구매날짜"])
test_ft["첫구매날짜"] = pd.to_datetime(test_ft["첫구매날짜"])
test_ft["마지막구매날짜"] = pd.to_datetime(test_ft["마지막구매날짜"])

train_ft["첫구매년도"] = train_ft["첫구매날짜"].dt.year
train_ft["첫구매월"] = train_ft["첫구매날짜"].dt.month
train_ft["마지막구매년도"] = train_ft["마지막구매날짜"].dt.year
train_ft["마지막구매월"] = train_ft["마지막구매날짜"].dt.month

test_ft["첫구매년도"] = test_ft["첫구매날짜"].dt.year
test_ft["첫구매월"] = test_ft["첫구매날짜"].dt.month
test_ft["마지막구매년도"] = test_ft["마지막구매날짜"].dt.year
test_ft["마지막구매월"] = test_ft["마지막구매날짜"].dt.month

train_ft.shape, test_ft.shape

((14940, 2363), (12225, 2363))

- 범주형 피처
    - 주구매지점, 주구매_중분류, 주구매_대분류, 주구매시간대, 선호브랜드코드, 고객등급

In [32]:
# %pip install category_encoders

In [33]:
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

oh_enc = OneHotEncoder(handle_unknown="ignore")
oh_enc_cols = ["주구매지점","주구매_대분류", "주구매시간대"]
oh_enc_data = pd.concat([train_ft[oh_enc_cols], test_ft[oh_enc_cols]])
oh_enc.fit(oh_enc_data)

train_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(train_ft[oh_enc_cols]).toarray()
test_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(test_ft[oh_enc_cols]).toarray()

train_ft.shape, test_ft.shape

((14940, 2404), (12225, 2404))

In [34]:
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

oh_enc = OneHotEncoder(handle_unknown="ignore")
oh_enc_cols = ["주구매지점", "주구매시간대"]
oh_enc_data = pd.concat([train_ft[oh_enc_cols], test_ft[oh_enc_cols]])
oh_enc.fit(oh_enc_data)

train_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(train_ft[oh_enc_cols]).toarray()
test_ft[oh_enc.get_feature_names_out()] = oh_enc.transform(test_ft[oh_enc_cols]).toarray()

train_ft.shape, test_ft.shape

((14940, 2404), (12225, 2404))

In [35]:
train_ft['선호브랜드코드'] = train_ft['선호브랜드코드'].astype('category')
test_ft['선호브랜드코드'] = test_ft['선호브랜드코드'].astype('category')

combined_brands = pd.concat([train_ft['선호브랜드코드'], test_ft['선호브랜드코드']])

enc = ce.CountEncoder()
train_ft["선호브랜드코드_cnt"] = enc.fit_transform(train_ft[['선호브랜드코드']])
test_ft["선호브랜드코드_cnt"] = enc.transform(test_ft[['선호브랜드코드']])

train_ft.shape, test_ft.shape

((14940, 2405), (12225, 2405))

In [36]:
combined_categories = pd.concat([train_ft['주구매_중분류'], test_ft['주구매_중분류']])

enc = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft.shape, test_ft.shape

((14940, 2406), (12225, 2406))

In [37]:
# train_ft와 test_ft의 열 이름을 각각 집합으로 변환
train_columns = set(train_ft.columns)
test_columns = set(test_ft.columns)

# 공통으로 있는 열
common_columns = train_columns.intersection(test_columns)
print("공통 변수:", common_columns)

# train_ft에만 있는 열
only_in_train = train_columns - test_columns
print("train_ft에만 있는 변수:", only_in_train)

# test_ft에만 있는 열
only_in_test = test_columns - train_columns
print("test_ft에만 있는 변수:", only_in_test)

공통 변수: {'대분류별_구매횟수_skew', 'pivot_브랜드코드_5873', 'pivot_브랜드코드_5537', 'pivot_브랜드코드_5575', 'pivot_브랜드코드_5825', 'pivot_브랜드코드_5181', 'pivot_브랜드코드_6360', 'pivot_브랜드코드_6473', 'pivot_브랜드코드_6234', 'pivot_브랜드코드_6921', 'pivot_브랜드코드_5231', 'pivot_브랜드코드_6544', 'pivot_브랜드코드_5727', 'pivot_브랜드코드_5750', 'Friday_sum', 'pivot_중분류_임대슈즈', 'pivot_브랜드코드_6775', 'pivot_브랜드코드_6818', 'pivot_브랜드코드_5323', 'pivot_브랜드코드_6876', 'pivot_브랜드코드_6002', 'pivot_브랜드코드_5160', '브랜드코드_nunique', 'pivot_브랜드코드_5153', 'pivot_브랜드코드_6136', 'pivot_브랜드코드_6140', 'pivot_브랜드코드_6135', 'pivot_브랜드코드_5669', 'pivot_중분류_냉장고.세탁기', 'pivot_브랜드코드_5742', '최대환불금액', 'pivot_브랜드코드_5159', 'pivot_브랜드코드_5368', 'pivot_브랜드코드_5528', 'pivot_브랜드코드_5333', 'pivot_중분류_모피_피혁', 'pivot_브랜드코드_6132', 'pivot_중분류_NB제화', 'pivot_브랜드코드_5678', 'pivot_브랜드코드_5946', '선호브랜드코드_cnt', 'pivot_브랜드코드_5507', 'pivot_브랜드코드_5183', '요일별_구매비율_std', 'pivot_브랜드코드_5595', 'pivot_브랜드코드_5733', 'pivot_브랜드코드_5633', 'pivot_브랜드코드_5478', 'pivot_브랜드코드_5647', 'pivot_브랜드코드_5782', 'pivot_브랜드코드_5828', 'pivot

In [38]:
## 고객등급 라벨 인코딩
#from sklearn.preprocessing import LabelEncoder

#le = LabelEncoder()
#train_ft["고객등급_인코딩"] = le.fit_transform(train_ft["고객등급"])

In [39]:
#grade_mapping = {'Diamond': 4, 'Platinum': 3, 'Gold': 2, 'Black': 1, 'Noraml': 0}
#train_ft["고객등급_인코딩"] = train_ft["고객등급"].map(grade_mapping)

In [40]:
#train_ft["고객등급"]

In [41]:
#train_ft["고객등급"] = train_ft["고객등급"].str.strip() #공백제거
#grade_mapping = {'Diamond': 4, 'Platinum': 3, 'Gold': 2, 'Black': 1, 'Noraml': 0}
#train_ft["고객등급_인코딩"] = train_ft["고객등급"].replace(grade_mapping)

In [42]:
#print(train_ft["고객등급"].unique())

In [43]:
## 대소문자 차이를 없애기 위해 모두 소문자로 변환
#train_ft["고객등급"] = train_ft["고객등급"].str.lower()

# 인코딩할 딕셔너리 정의 (소문자에 맞추어 변경)
#grade_mapping = {'diamond': 4, 'platinum': 3, 'gold': 2, 'black': 1, 'normal': 0}

# "고객등급"을 딕셔너리를 통해 인코딩
#train_ft["고객등급_인코딩"] = train_ft["고객등급"].map(grade_mapping)

In [44]:
#train_ft["고객등급_인코딩"]

In [45]:
#print(train_ft["고객등급_인코딩"].unique())

In [46]:
#diamond_count = (train_ft["고객등급"] == "Diamond").sum()
#print(diamond_count)

In [47]:
#diamond_count = (train_ft["고객등급_인코딩"] == 4).sum()
#print(diamond_count)

In [48]:
#diamond_count = (train_ft["고객등급"] == "Platinum").sum()
#print(diamond_count)

In [49]:
#diamond_count = (train_ft["고객등급"] == "Gold").sum()
#print(diamond_count)

In [50]:
#diamond_count = (train_ft["고객등급"] == "Black").sum()
#print(diamond_count)

In [51]:
#diamond_count = (train_ft["고객등급"] == "Normal").sum()
#print(diamond_count)

In [52]:
## NaN 값의 갯수 확인
#nan_count = train_ft["고객등급"].isna().sum()
#print(nan_count)

In [53]:
#test_ft["고객등급"]

In [54]:
# 대소문자 차이를 없애기 위해 모두 소문자로 변환
#test_ft["고객등급"] = test_ft["고객등급"].str.lower()

# 인코딩할 딕셔너리 정의 (소문자에 맞추어 변경)
#grade_mapping = {'diamond': 4, 'platinum': 3, 'gold': 2, 'black': 1, 'normal': 0}

# "고객등급"을 딕셔너리를 통해 인코딩
#test_ft["고객등급_인코딩"] = test_ft["고객등급"].map(grade_mapping)

In [55]:
#test_ft["고객등급_인코딩"]

## 원핫인코딩

In [56]:
#import pandas as pd
#from sklearn.preprocessing import OneHotEncoder

# 예시 데이터프레임
# test_ft = pd.DataFrame({'고객등급': ['diamond', 'normal', 'gold', 'platinum', 'black']})

# OneHotEncoder 객체 생성 (drop='first'를 설정하면 첫 번째 범주를 제외한 나머지를 인코딩하여 다중 공선성 문제를 줄일 수 있음)
#encoder = OneHotEncoder(sparse=False)

# 고객등급 컬럼을 2차원 배열로 변환하여 인코딩 수행
#encoded_data = encoder.fit_transform(train_ft[["고객등급"]])

# 인코딩된 데이터를 데이터프레임으로 변환하여 기존 컬럼명에 맞춰 열 이름 지정
#encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(["고객등급"]))

# 원본 데이터프레임과 인코딩된 데이터프레임 병합
#train_ft = pd.concat([train_ft, encoded_df], axis=1).drop(columns=["고객등급"])

# 결과 출력
#print(train_ft)

In [57]:
#train_ft["고객등급"]

In [58]:
import pandas as pd

# 원핫인코딩 수행
train_ft = pd.get_dummies(train_ft, columns=['고객등급'], prefix='고객등급')

# 원핫인코딩된 열만 선택하여 True/False를 1/0으로 변환
for col in train_ft.filter(like='고객등급_').columns:
    train_ft[col] = train_ft[col].astype(int)

# 결과 확인
train_ft.head()

총방문일수      첫구매날짜    마지막구매날짜  백화점이용기간  구매주기  주말방문일수  일별평균구매건수  구매간격_표준편차  \
0      9 2004-05-07 2005-01-16      255    28       2  1.555556  22.744789   
1     21 2004-05-11 2005-04-27      352    16       1  2.000000  11.739134   
2     55 2004-05-06 2005-04-26      356     6      11  1.854545   5.019987   
3     90 2004-05-11 2005-04-20      345     3      19  2.055556   3.137146   
4     24 2004-05-09 2005-03-17      313    13       6  1.500000  11.962731   

     평균구매요일  주구매요일  ...  주구매시간대_18  주구매시간대_19  주구매시간대_20  선호브랜드코드_cnt  \
0  3.000000      2  ...        0.0        0.0        0.0            1   
1  2.309524      3  ...        1.0        0.0        0.0         7051   
2  2.333333      0  ...        0.0        0.0        0.0            8   
3  3.081081      4  ...        0.0        0.0        0.0          215   
4  3.861111      4  ...        0.0        0.0        0.0         7051   

   주구매_중분류_cnt  고객등급_Black  고객등급_Diamond  고객등급_Gold  고객등급_Normal  \
0          236           0             0          0            1   
1          521           0             0          0            1   
2           14           0             1          0            0   
3         4669           0             1          0            0   
4         2049           0             0          0            1   

   고객등급_Platinum  
0              0  
1              0  
2              0  
3              0  
4              0  

[5 rows x 2410 columns]

In [59]:
# 원핫인코딩 수행
test_ft = pd.get_dummies(test_ft, columns=['고객등급'], prefix='고객등급')

# 원핫인코딩된 열만 선택하여 True/False를 1/0으로 변환
for col in test_ft.filter(like='고객등급_').columns:
    test_ft[col] = test_ft[col].astype(int)

# 결과 확인
test_ft.head()

총방문일수      첫구매날짜    마지막구매날짜  백화점이용기간  구매주기  주말방문일수  일별평균구매건수  구매간격_표준편차  \
0      5 2004-05-16 2004-10-26      164    32       3  1.400000  30.416096   
1      3 2004-09-11 2005-02-23      166    55       1  1.333333  57.879185   
2      2 2004-08-16 2004-09-02       18     9       0  2.500000   7.361216   
3     14 2004-07-22 2005-04-19      272    19       1  1.714286  17.715031   
4      7 2004-05-25 2005-02-13      265    37       3  2.142857  36.517818   

     평균구매요일  주구매요일  ...  주구매시간대_18  주구매시간대_19  주구매시간대_20  선호브랜드코드_cnt  \
0  3.428571      1  ...        0.0        0.0        0.0       7051.0   
1  1.750000      0  ...        0.0        0.0        0.0        284.0   
2  1.200000      0  ...        0.0        0.0        0.0        577.0   
3  3.000000      3  ...        1.0        0.0        0.0          2.0   
4  4.000000      6  ...        0.0        1.0        0.0       7051.0   

   주구매_중분류_cnt  고객등급_Black  고객등급_Diamond  고객등급_Gold  고객등급_Normal  \
0       2049.0           0             0          0            1   
1         25.0           0             0          0            1   
2       4669.0           0             0          0            1   
3        521.0           0             0          0            1   
4         13.0           0             0          0            1   

   고객등급_Platinum  
0              0  
1              0  
2              0  
3              0  
4              0  

[5 rows x 2410 columns]

- 문자열 피처 삭제

In [60]:
cols = train_ft.select_dtypes(["object", "datetime"]).columns.tolist()
cols

['첫구매날짜', '마지막구매날짜', '주구매지점', '주구매_중분류', '주구매_대분류']

In [61]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 2405), (12225, 2405))

In [62]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

In [63]:
train_ft.select_dtypes("datetime").columns, test_ft.select_dtypes("datetime").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

In [64]:
train_ft.isin([np.inf, -np.inf]).any().sum(), test_ft.isin([np.inf, -np.inf]).any().sum()

(0, 0)

In [65]:
train_ft.isna().sum().sum(), test_ft.isna().sum().sum()

(0, 0)

In [66]:
import pandas as pd

# NaN이 포함된 컬럼 찾기
nan_columns = [col for col in train_ft.columns if train_ft[col].isnull().any()]

print("NaN이 포함된 컬럼:", nan_columns)

NaN이 포함된 컬럼: []


In [67]:
#train_ft=train_ft.drop(["브랜드코드별_구매횟수_std", "브랜드코드별_구매횟수_skew", "브랜드코드별_구매횟수_kurt"], axis=1)
#test_ft=test_ft.drop(["브랜드코드별_구매횟수_std", "브랜드코드별_구매횟수_skew", "브랜드코드별_구매횟수_kurt"], axis=1)

In [68]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer, Normalizer
scaler = Normalizer()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

총방문일수   백화점이용기간          구매주기        주말방문일수      일별평균구매건수     구매간격_표준편차  \
0  0.000001  0.000041  4.547622e-06  3.248301e-07  2.526457e-07  3.694097e-06   
1  0.000003  0.000059  2.663584e-06  1.664740e-07  3.329480e-07  1.954261e-06   
2  0.000002  0.000014  2.279968e-07  4.179942e-07  7.047175e-08  1.907569e-07   
3  0.000002  0.000008  6.835463e-08  4.329127e-07  4.683558e-08  7.147948e-08   
4  0.000003  0.000036  1.493358e-06  6.892421e-07  1.723105e-07  1.374203e-06   

         평균구매요일         주구매요일      월요일_구매비율      화요일_구매비율  ...     주구매시간대_18  \
0  4.872452e-07  3.248301e-07  1.160108e-08  0.000000e+00  ...  0.000000e+00   
1  3.844757e-07  4.994221e-07  7.927334e-09  4.360034e-08  ...  1.664740e-07   
2  8.866543e-08  0.000000e+00  9.686140e-09  7.078333e-09  ...  0.000000e+00   
3  7.020206e-08  9.113951e-08  1.724261e-09  2.586392e-09  ...  0.000000e+00   
4  4.435400e-07  4.594947e-07  9.572807e-09  6.381871e-09  ...  0.000000e+00   

   주구매시간대_19  주구매시간대_20   선호브랜드코드_cnt   주구매_중분류_cnt  고객등급_Black  고객등급_Diamond  \
0        0.0        0.0  1.624151e-07  3.832996e-05         0.0  0.000000e+00   
1        0.0        0.0  1.173808e-03  8.673296e-05         0.0  0.000000e+00   
2        0.0        0.0  3.039958e-07  5.319926e-07         0.0  3.799947e-08   
3        0.0        0.0  4.898749e-06  1.063826e-04         0.0  2.278488e-08   
4        0.0        0.0  8.099743e-04  2.353762e-04         0.0  0.000000e+00   

   고객등급_Gold   고객등급_Normal  고객등급_Platinum  
0        0.0  1.624151e-07            0.0  
1        0.0  1.664740e-07            0.0  
2        0.0  0.000000e+00            0.0  
3        0.0  0.000000e+00            0.0  
4        0.0  1.148737e-07            0.0  

[5 rows x 2405 columns]

In [69]:
train_ft

총방문일수   백화점이용기간          구매주기        주말방문일수      일별평균구매건수  \
0      1.461736e-06  0.000041  4.547622e-06  3.248301e-07  2.526457e-07   
1      3.495954e-06  0.000059  2.663584e-06  1.664740e-07  3.329480e-07   
2      2.089971e-06  0.000014  2.279968e-07  4.179942e-07  7.047175e-08   
3      2.050639e-06  0.000008  6.835463e-08  4.329127e-07  4.683558e-08   
4      2.756968e-06  0.000036  1.493358e-06  6.892421e-07  1.723105e-07   
...             ...       ...           ...           ...           ...   
14935  5.591171e-06  0.000152  1.677351e-05  1.242482e-06  1.173456e-06   
14936  9.588027e-07  0.000056  1.390264e-05  2.397007e-07  2.397007e-07   
14937  1.087925e-05  0.000292  2.357170e-05  1.813208e-06  1.813208e-06   
14938  2.387476e-06  0.000068  5.570778e-06  3.979127e-07  5.471300e-07   
14939  3.818270e-06  0.000097  9.545675e-06  3.818270e-07  6.872886e-07   

          구매간격_표준편차        평균구매요일         주구매요일      월요일_구매비율      화요일_구매비율  \
0      3.694097e-06  4.872452e-07  3.248301e-07  1.160108e-08  0.000000e+00   
1      1.954261e-06  3.844757e-07  4.994221e-07  7.927334e-09  4.360034e-08   
2      1.907569e-07  8.866543e-08  0.000000e+00  9.686140e-09  7.078333e-09   
3      7.147948e-08  7.020206e-08  9.113951e-08  1.724261e-09  2.586392e-09   
4      1.374203e-06  4.435400e-07  4.594947e-07  9.572807e-09  6.381871e-09   
...             ...           ...           ...           ...           ...   
14935  1.249494e-05  1.936811e-06  1.242482e-06  3.654360e-08  0.000000e+00   
14936  5.108618e-06  9.588027e-07  9.588027e-07  0.000000e+00  0.000000e+00   
14937  1.978243e-05  1.586557e-06  9.066040e-07  3.777517e-08  4.155268e-07   
14938  4.522938e-06  6.089271e-07  5.968691e-07  0.000000e+00  4.823185e-08   
14939  8.935688e-06  1.293969e-06  1.145481e-06  0.000000e+00  0.000000e+00   

       ...     주구매시간대_18     주구매시간대_19  주구매시간대_20   선호브랜드코드_cnt   주구매_중분류_cnt  \
0      ...  0.000000e+00  0.000000e+00        0.0  1.624151e-07  3.832996e-05   
1      ...  1.664740e-07  0.000000e+00        0.0  1.173808e-03  8.673296e-05   
2      ...  0.000000e+00  0.000000e+00        0.0  3.039958e-07  5.319926e-07   
3      ...  0.000000e+00  0.000000e+00        0.0  4.898749e-06  1.063826e-04   
4      ...  0.000000e+00  0.000000e+00        0.0  8.099743e-04  2.353762e-04   
...    ...           ...           ...        ...           ...           ...   
14935  ...  0.000000e+00  0.000000e+00        0.0  1.950697e-04  7.020025e-05   
14936  ...  0.000000e+00  2.397007e-07        0.0  1.869665e-05  2.397007e-07   
14937  ...  0.000000e+00  0.000000e+00        0.0  6.392465e-03  5.711605e-05   
14938  ...  0.000000e+00  0.000000e+00        0.0  2.984345e-06  1.036563e-04   
14939  ...  0.000000e+00  0.000000e+00        0.0  7.636540e-07  3.321895e-05   

       고객등급_Black  고객등급_Diamond  고객등급_Gold   고객등급_Normal  고객등급_Platinum  
0             0.0  0.000000e+00        0.0  1.624151e-07            0.0  
1             0.0  0.000000e+00        0.0  1.664740e-07            0.0  
2             0.0  3.799947e-08        0.0  0.000000e+00            0.0  
3             0.0  2.278488e-08        0.0  0.000000e+00            0.0  
4             0.0  0.000000e+00        0.0  1.148737e-07            0.0  
...           ...           ...        ...           ...            ...  
14935         0.0  0.000000e+00        0.0  6.212412e-07            0.0  
14936         0.0  0.000000e+00        0.0  2.397007e-07            0.0  
14937         0.0  0.000000e+00        0.0  9.066040e-07            0.0  
14938         0.0  0.000000e+00        0.0  1.989564e-07            0.0  
14939         0.0  0.000000e+00        0.0  3.818270e-07            0.0  

[14940 rows x 2405 columns]

In [70]:
train_ft.head()

총방문일수   백화점이용기간          구매주기        주말방문일수      일별평균구매건수     구매간격_표준편차  \
0  0.000001  0.000041  4.547622e-06  3.248301e-07  2.526457e-07  3.694097e-06   
1  0.000003  0.000059  2.663584e-06  1.664740e-07  3.329480e-07  1.954261e-06   
2  0.000002  0.000014  2.279968e-07  4.179942e-07  7.047175e-08  1.907569e-07   
3  0.000002  0.000008  6.835463e-08  4.329127e-07  4.683558e-08  7.147948e-08   
4  0.000003  0.000036  1.493358e-06  6.892421e-07  1.723105e-07  1.374203e-06   

         평균구매요일         주구매요일      월요일_구매비율      화요일_구매비율  ...     주구매시간대_18  \
0  4.872452e-07  3.248301e-07  1.160108e-08  0.000000e+00  ...  0.000000e+00   
1  3.844757e-07  4.994221e-07  7.927334e-09  4.360034e-08  ...  1.664740e-07   
2  8.866543e-08  0.000000e+00  9.686140e-09  7.078333e-09  ...  0.000000e+00   
3  7.020206e-08  9.113951e-08  1.724261e-09  2.586392e-09  ...  0.000000e+00   
4  4.435400e-07  4.594947e-07  9.572807e-09  6.381871e-09  ...  0.000000e+00   

   주구매시간대_19  주구매시간대_20   선호브랜드코드_cnt   주구매_중분류_cnt  고객등급_Black  고객등급_Diamond  \
0        0.0        0.0  1.624151e-07  3.832996e-05         0.0  0.000000e+00   
1        0.0        0.0  1.173808e-03  8.673296e-05         0.0  0.000000e+00   
2        0.0        0.0  3.039958e-07  5.319926e-07         0.0  3.799947e-08   
3        0.0        0.0  4.898749e-06  1.063826e-04         0.0  2.278488e-08   
4        0.0        0.0  8.099743e-04  2.353762e-04         0.0  0.000000e+00   

   고객등급_Gold   고객등급_Normal  고객등급_Platinum  
0        0.0  1.624151e-07            0.0  
1        0.0  1.664740e-07            0.0  
2        0.0  0.000000e+00            0.0  
3        0.0  0.000000e+00            0.0  
4        0.0  1.148737e-07            0.0  

[5 rows x 2405 columns]

# 차원축소

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import StratifiedKFold, cross_val_score
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier

#cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

#model = LGBMClassifier(random_state=SEED)
#model.fit(train_ft_tmp_col, target)

#scores = cross_val_score(model, train_ft_tmp_col, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores)

- XAI 활용

In [ ]:
#%pip install shap

In [ ]:
#%pip install numpy==2.0

In [ ]:
#import shap

#explainer = shap.Explainer(model)
#shap_values = explainer(train_ft_tmp)
#shap.summary_plot(shap_values, train_ft_tmp)

- SelectFromModel

In [ ]:
#from sklearn.feature_selection import SelectFromModel

#rf = RandomForestClassifier(random_state=SEED)

## thresholds = [
##     'mean', 'median',
##     '0.5*mean', '0.8*mean', '1.2*mean', '1.5*mean', '1.8*mean', '2*mean',
##     '0.5*median', '0.8*median', '1.2*median', '1.5*median', '1.8*median', '2*median'
## ]

# thresholds = [f"{i}*mean" for i in np.arange(0.1, 1.1, 0.1)] # 0.5*mean 일 때 성능이 가장 높음
# thresholds = [f"{i}*median" for i in np.arange(1.5, 3.6, 0.1)] # 2.5*median
# thresholds = [f"{i}*median" for i in np.arange(3.5, 4.0, 0.1)] # 3.5*median
#thresholds = ['2.5*median', '3.5*median', '0.5*mean']

#for threshold in thresholds:
#    sfm = SelectFromModel(rf, threshold=threshold)
#    train_sfm = sfm.fit_transform(train_ft_tmp_col, target)

#    scores = cross_val_score(model, train_sfm, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#    print(f"threshold={threshold}, 성능={np.mean(scores)}, 특성 수={train_sfm.shape[1]}")

In [ ]:
#sfm = SelectFromModel(rf, threshold='3.5*median')
#train_sfm = sfm.fit_transform(train_ft_tmp_col, target)

#scores = cross_val_score(model, train_sfm, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), train_sfm.shape

- SelectKBest

In [ ]:
#from sklearn.feature_selection import SelectKBest, VarianceThreshold

#vt = VarianceThreshold(threshold=0)
#train_vt = vt.fit_transform(train_ft_tmp_col)
#test_vt = vt.transform(test_ft_tmp_col)

In [ ]:
#for k in range(300, 1001, 50):
#    skb = SelectKBest(k=k)
#    train_skb = skb.fit_transform(train_vt, target)

#    scores = cross_val_score(model, train_skb, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#    print(f"특성수={k}, 정확도={np.mean(scores)}")

In [ ]:
#for k in range(590, 610):
#    skb = SelectKBest(k=k)
#    train_skb = skb.fit_transform(train_vt, target)

#    scores = cross_val_score(model, train_skb, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#    print(f"특성수={k}, 정확도={np.mean(scores)}")

In [ ]:
#skb = SelectKBest(k=600)
#train_skb = skb.fit_transform(train_vt, target)

#scores = cross_val_score(model, train_skb, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), train_skb.shape

- SelectPercentile

In [ ]:
#from sklearn.feature_selection import SelectPercentile

# percentiles = [i for i in range(1, 100, 5)] # 16 ~ 26 / 26 ~ 41 / 56 ~ 66 / 86 이상에서 성능점수 높게나옴
#percentiles = [i for i in range(56, 67)] # 24일 때 성능점수 가장 높음

#for percentile in percentiles:
#    sp = SelectPercentile(percentile=percentile)
#    train_sp = sp.fit_transform(train_vt, target)

#    scores = cross_val_score(model, train_sp, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#    print(f"percentile={percentile}, 성능={np.mean(scores)}, 특성 수={train_sp.shape[1]}")

In [ ]:
#sp = SelectPercentile(percentile=24)
#train_sp = sp.fit_transform(train_vt, target)

#scores = cross_val_score(model, train_sp, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), train_sp.shape

- SelectPercentile 에서 구한 값 근처의 피처 개수에 대해 SelectKBest

In [ ]:
#for k in range(550, 561):
#    skb = SelectKBest(k=k)
#    train_skb = skb.fit_transform(train_sp, target)

#    scores = cross_val_score(model, train_skb, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#    print(f"특성수={k}, 정확도={np.mean(scores)}") # 특성수 569 일 때 최고성능점수

In [ ]:
#skb = SelectKBest(k=569)
#train_skb = skb.fit_transform(train_vt, target)

#scores = cross_val_score(model, train_skb, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), train_skb.shape

- PCA

In [ ]:
#from sklearn.decomposition import PCA

#pca = PCA(train_ft.shape[1], random_state=SEED)
#pca_train = pca.fit_transform(train_ft_tmp_col)

#scores = cross_val_score(model, pca_train, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), pca_train.shape

In [ ]:
#from sklearn.decomposition import PCA

## n_components를 피처 수보다 작은 값으로 설정
#pca = PCA(n_components=0.95, random_state=SEED)  # 데이터의 95% 분산 설명
#pca_train = pca.fit_transform(train_ft_tmp)

#scores = cross_val_score(model, pca_train, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), pca_train.shape

In [ ]:
#cumsum = np.cumsum(pca.explained_variance_ratio_)
#n_components = np.argmax(cumsum >= 0.7) + 1
#n_components

In [ ]:
#pca = PCA(n_components, random_state=SEED)
#x_train = pca.fit_transform(train_ft_tmp)

#scores = cross_val_score(model, x_train, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), x_train.shape

- LDA

In [ ]:
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
#from sklearn.model_selection import train_test_split

#X_train, X_val, y_train, y_val = train_test_split(
#    train_ft_tmp_col, target, test_size=0.2, random_state=SEED
#)

#lda = LDA(n_components=1)
#X_train_lda = lda.fit_transform(X_train, y_train)
#X_val_lda = lda.transform(X_val)

#scores = cross_val_score(model, X_train_lda, y_train, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), X_train_lda.shape

In [ ]:
#scores = cross_val_score(model, X_val_lda, y_val, cv=cv, scoring='f1_macro', n_jobs=-1)
#np.mean(scores)

- RFE

In [ ]:

#from sklearn.feature_selection import RFE

#rfe = RFE(estimator=model, n_features_to_select=500, step=50)
#x_train_rfe = rfe.fit_transform(train_ft_tmp_col, target)

#scores = cross_val_score(model, x_train_rfe, target, cv=cv, scoring="f1_macro", n_jobs=-1)
#np.mean(scores), x_train_rfe.shape

- UMAP

In [ ]:
#%pip install umap-learn

In [ ]:
#import numpy as np
#from sklearn.model_selection import cross_val_score
#import umap
#from sklearn.ensemble import RandomForestClassifier  # 예시 모델
#from sklearn.datasets import make_classification

## 예시 데이터 (사용 중인 train_ft와 target을 여기에 할당)
#train_ft, target = make_classification(n_samples=1000, n_features=1000, random_state=42)

## 1. UMAP 설정 및 변환 (차원을 500으로 줄임)
#reducer = umap.UMAP(n_components=500, random_state=42)
#x_train_umap = reducer.fit_transform(train_ft_tmp_col)

## 2. 모델 정의
#model = RandomForestClassifier(random_state=42)

## 3. 교차 검증 (F1 매크로 스코어 평가)
#scores = cross_val_score(model, x_train_umap, target, cv=5, scoring="f1_macro", n_jobs=-1)
#print("Average F1 Macro Score:", np.mean(scores))
#print("Transformed Shape:", x_train_umap.shape)

- 최종적으로 모델에 넣을 학습데이터

In [ ]:
#skb = SelectKBest(k=734) # 569 or 726
#train_ft_tmptmp = skb.fit_transform(train_vt, target)
#test_ft_tmptmp = skb.transform(test_vt)
#train_ft_tmptmp.shape, test_ft_tmptmp.shape

In [ ]:
#sp = SelectPercentile(percentile=65)
#train_sp = sp.fit_transform(train_vt, target)
#test_sp = sp.transform(test_vt)
#train_sp.shape, test_sp.shape

# 피처 셀렉션

In [ ]:
#from sklearn.feature_selection import RFE
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import train_test_split, cross_val_score
#import numpy as np#

## 데이터 준비 (예시 데이터셋 사용)
#X, y = load_data(train_ft, test_ft)  # load_data() 함수는 예시로, 실제 데이터셋을 여기에 넣으세요.
#model = RandomForestClassifier()  # 사용할 모델 설정 (임의로 RandomForestClassifier 사용)#

## RFE 설정 - 처음에는 모든 피처를 포함시킨 후, 중요도가 낮은 피처를 하나씩 제거
#rfe = RFE(estimator=model, n_features_to_select=5, step=1)  # 최종적으로 5개의 피처가 남을 때까지 진행
#X_rfe = rfe.fit_transform(X, y)#

## 최적 피처 확인
#selected_features = np.array(X.columns)[rfe.support_]  # 선택된 피처 목록
#print("선택된 피처:", selected_features)#

## 교차 검증으로 모델 성능 확인
#scores = cross_val_score(model, X_rfe, y, cv=5, scoring="f1_macro")
#print("평균 F1 스코어:", np.mean(scores))

In [ ]:
#train_ft.columns

In [ ]:
## 실습을 위한 임의의 분류용 데이터 생성
#from sklearn.datasets import make_classification#

#X, y = make_classification(n_samples=14940, n_features=2379, n_informative=3,
#                          n_redundant=2, n_repeated=0, n_classes=8,
#                          n_clusters_per_class=1, random_state=0)#

## 피처들을 데이터프레임 형태로 만들어보자
#import pandas as pd#

#columns_list = [train_ft.columns]
#df_X = pd.DataFrame(X, columns=columns_list)

In [ ]:
##REF
#from sklearn.feature_selection import RFE
#from sklearn.svm import SVC # 사용할 모델

#svc = SVC(kernel='linear')

#rfe = RFE(estimator=svc, n_features_to_select=100, step=1) # step=1: 하나씩 피처 제거, n_features_to_select=5: 5개 피처를 선택
#rfe.fit(X, y)

In [ ]:
#print(rfe.support_) # 선택된 피처는 True로 반환
#print('선택된 피처:', df_X.columns[rfe.support_])

In [ ]:
## 선택된 피처를 통한 최종 데이터프레임 생성
#feature_selection = df_X.columns[rfe.support_]
#df_X = df_X[feature_selection]
#df = df_X # 데이터프레임 이름 변경
#df['target'] = y

#df.head()

In [ ]:
#train_ft = df

In [ ]:
#train_ft.shape

# 피처 상관관계 분석

In [ ]:
# 상관관계 계산
correlation_matrix = train_ft.corr()
print(correlation_matrix)

# 상관관계가 높은 변수들 제거 (예: 0.9 이상)
upper_triangle = correlation_matrix.where(abs(correlation_matrix) > 0.8)
print(upper_triangle)

In [ ]:
#import seaborn as sns
#import matplotlib.pyplot as plt
#from tqdm import tqdm
#import time

## 100번 반복하면서 진행 상황을 표시
#for i in tqdm(range(100)):
#    time.sleep(0.1)  # 각 아이템 처리마다 0.1초 지연
    
# 상관 행렬 계산
#correlation_matrix = train_ft.corr()

## 상관 행렬을 히트맵으로 시각화
#plt.figure(figsize=(12, 10))  # 그래프 크기 설정
#sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
#plt.title('Correlation Matrix')
#plt.show()

In [ ]:
#from tqdm import tqdm
#import time
#import matplotlib.pyplot as plt
#import seaborn as sns

## 100번 반복하면서 진행 상황을 표시
#for i in tqdm(range(100)):
#    time.sleep(0.1)  # 각 아이템 처리마다 0.1초 지연
    
## 상관계수가 0.9 이상인 상관 행렬 추출
#upper_triangle = correlation_matrix.where(abs(correlation_matrix) > 0.9)

## 상관 행렬을 히트맵으로 시각화 (상관계수가 0.9 이상인 것만)
#plt.figure(figsize=(12, 10))  # 그래프 크기 설정
#sns.heatmap(upper_triangle, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
#plt.title('High Correlation (>= 0.9)')
#plt.show()

In [ ]:
## 0.9 이상인 상관계수 컬럼 출력
#high_correlation_columns = upper_triangle.dropna(how='all', axis=0).dropna(how='all', axis=1)
#print("0.9 이상인 상관계수가 있는 컬럼들:")
#print(high_correlation_columns)

In [ ]:
## 0.9 이상이면서 1.00 미만인 상관계수 컬럼들을 리스트로 추출
#high_correlation_pairs = []

#for column in upper_triangle.columns:
#    for row in upper_triangle.index:
#        # 상관계수가 0.9 이상이고 1.00 미만이며, 대각선이 아닌 경우만 추가
#        if not pd.isna(upper_triangle.loc[row, column]) and row != column and 0.9 <= upper_triangle.loc[row, column] < 0.99:
#            high_correlation_pairs.append((row, column, upper_triangle.loc[row, column]))

## 내림차순으로 정렬
#high_correlation_pairs = sorted(high_correlation_pairs, key=lambda x: x[2], reverse=True)

## 출력: 0.9 이상이면서 1.00 미만인 상관계수가 있는 컬럼 쌍과 상관계수
#print("0.9 이상이면서 1.00 미만인 상관계수가 있는 컬럼 쌍들 (내림차순 정렬):")
#for pair in high_correlation_pairs:
#    print(f"{pair[0]} and {pair[1]}: {pair[2]:.2f}")

 #   print(f"{pair[0]} and {pair[1]}: {pair[2]:.2f}")

In [ ]:
# 0.9 이상이면서 1.00 미만인 상관계수 컬럼들을 리스트로 추출
high_correlation_pairs = []

for column in upper_triangle.columns:
    for row in upper_triangle.index:
        # 상관계수가 0.9 이상이고 1.00 미만이며, 대각선이 아닌 경우만 추가
        if not pd.isna(upper_triangle.loc[row, column]) and row != column and 0.8 <= upper_triangle.loc[row, column] < 1.0:
            high_correlation_pairs.append((row, column, upper_triangle.loc[row, column]))

# 내림차순으로 정렬
high_correlation_pairs = sorted(high_correlation_pairs, key=lambda x: x[2], reverse=True)

# 문자열 리스트로 저장
correlation_strings = [f"{pair[0]} and {pair[1]}: {pair[2]:.2f}" for pair in high_correlation_pairs]

# 리스트 출력
print(correlation_strings)

In [ ]:
import pandas as pd

# 0.9 이상이면서 1.00 미만인 상관계수 컬럼들을 리스트로 추출
high_correlation_pairs = []

for column in upper_triangle.columns:
    for row in upper_triangle.index:
        # 상관계수가 0.9 이상이고 1.00 미만이며, 대각선이 아닌 경우만 추가
        if not pd.isna(upper_triangle.loc[row, column]) and row != column and 0.9 <= upper_triangle.loc[row, column] < 1.0:
            high_correlation_pairs.append((row, column, upper_triangle.loc[row, column]))

# 내림차순으로 정렬
high_correlation_pairs = sorted(high_correlation_pairs, key=lambda x: x[2], reverse=True)

# DataFrame으로 변환
correlation_df = pd.DataFrame(high_correlation_pairs, columns=['Variable 1', 'Variable 2', 'Correlation'])

# 엑셀 파일로 저장
correlation_df.to_excel("high_correlation_pairs_3.xlsx", index=False)

print("상관관계 쌍이 엑셀 파일로 저장되었습니다.")

In [ ]:
train_ft["마지막구매년도"]

In [ ]:
train_ft

In [ ]:
#train_ft_tmp = train_ft.drop(columns=["첫구매년도", "마지막구매년도",  "pivot_중분류_색조화장품", "월별_구매비율_skew", "pivot_브랜드코드_5208", "pivot_중분류_홈데코", "pivot_중분류_피혁","계절_구매변동성", "pivot_중분류_유니섹스캐주얼", "pivot_중분류_수입캐주얼"  ])


In [ ]:
#test_ft_tmp = test_ft.drop(columns=["첫구매년도", "마지막구매년도",  "pivot_중분류_색조화장품", "월별_구매비율_skew", "pivot_브랜드코드_5208", "pivot_중분류_홈데코", "pivot_중분류_피혁","계절_구매변동성", "pivot_중분류_유니섹스캐주얼", "pivot_중분류_수입캐주얼"  ])


#상관계수가 0.9 이상이면 두 변수 간에 매우 강한 상관관계가 있다는 뜻입니다. 이렇게 높은 상관관계는 다음과 같은 이유로 머신러닝 모델에 부정적인 영향을 줄 수 있습니다:

1. 다중공선성 (Multicollinearity) 문제
상관계수가 높은 피처들은 서로 유사한 정보를 포함하고 있어서, 두 피처 중 하나는 불필요한 정보일 가능성이 높습니다. 특히 선형 모델(예: 선형 회귀)에서는 다중공선성이 심해질 수 있으며, 이로 인해 모델의 예측 안정성이 떨어지고 가중치 추정이 불안정해집니다.
다중공선성 문제는 회귀 분석이나 로지스틱 회귀와 같은 선형 모델의 성능을 저하시킬 수 있고, 모델이 각 변수의 기여도를 정확히 파악하기 어려워집니다.
2. 과적합 (Overfitting) 위험 증가
두 피처가 거의 같은 정보를 담고 있을 때, 모델이 불필요한 피처를 모두 사용하게 되어 복잡성이 증가할 수 있습니다. 이는 모델이 학습 데이터에 과적합하여 새로운 데이터에 대한 일반화 성능이 떨어지게 할 수 있습니다.
높은 상관관계의 피처를 제거하면, 모델이 더 간결해지고 새로운 데이터에 대한 성능이 높아질 가능성이 있습니다.
3. 해석성 (Interpretability) 저하
머신러닝 모델의 피처들이 서로 독립적일 때, 결과 해석이 더 명확해집니다. 하지만 높은 상관관계를 가진 피처들이 모델에 포함되어 있으면, 각 피처의 영향을 명확히 해석하기 어려워지고, 중요한 피처들을 파악하는 데 혼란을 줄 수 있습니다.
4. 모델 효율성 저하
상관관계가 높은 피처를 모두 포함하면 모델이 불필요한 계산을 수행하게 됩니다. 특히 대용량 데이터에서는 이러한 불필요한 피처들이 학습 속도를 저하시키고, 메모리 사용량을 증가시킬 수 있습니다.
따라서 상관관계가 높은 피처들을 적절히 제거하면 모델이 더 효율적이고 빠르게 작동할 수 있습니다.
어떻게 해결하나요?
상관관계가 높은 피처를 하나 선택하여 나머지 피처는 제거하는 것이 일반적입니다.
피처 중요도를 평가하여 상관관계가 높은 피처들 중 중요한 하나만 남기고 제거하는 방법도 있습니다.
또한, 주성분 분석(PCA) 같은 차원 축소 기법을 사용하여 상관관계가 높은 피처들을 요약할 수 있습니다.
이처럼, 상관계수가 0.9 이상인 피처들은 일반적으로 하나만 남기고 제거하여 모델의 효율성과 성능을 높일 수 있습니다.

# 0.9

In [71]:
train_ft_tmp = train_ft.drop(columns=["pivot_브랜드코드_5171",
"pivot_중분류_색조화장품",
"첫구매년도",
"선호브랜드코드",
"마지막구매년도",
"브랜드코드별_구매횟수_kurt",
"선호브랜드코드_구매비율",
"평균구매시간",
"주구매시간대",
"월별_구매비율_skew",
"월별_구매비율_std",
"성수기_구매집중도",
"전체_성수기_구매비율",
"pivot_브랜드코드_5208",
"pivot_중분류_홈데코",
"pivot_브랜드코드_6829",
"pivot_중분류_피혁",
"pivot_브랜드코드_6995",
"pivot_브랜드코드_6803",
"계절_구매변동성",
"계절별_구매비율_std",
"분기별_구매비율_std",
"브랜드코드별_구매횟수_skew",
"중분류별_구매횟수_skew",
"주말_방문_선호도",
"주말_구매비율",
"대분류별_구매횟수_skew",
"요일별_구매비율_std",
"pivot_브랜드코드_5433",
"pivot_중분류_유니섹스캐주얼",
"pivot_브랜드코드_5464",
"pivot_중분류_수입캐주얼",
"pivot_브랜드코드_6735",
"pivot_브랜드코드_6506",
"중분류별_구매횟수_kurt",
"주구매_중분류_화장품",
"요일별_구매비율_skew",
"주구매지점_이용비율",
"오후_구매비율",
"구매간격_평균_x",
"구매간격_평균_y",
"구매간격_중앙값_x",
"구매간격_중앙값_y",
"지점코드별_구매횟수_skew",
"pivot_브랜드코드_5962",
"pivot_중분류_골프(용품)",
"대분류별_구매횟수_kurt",
"pivot_브랜드코드_6377",
"pivot_브랜드코드_6053",
"구매간격_최대값_x",
"구매간격_최대값_y",
"구매주기",
"구매금액_median",
"평균구매금액",
"월별_구매비율_kurt",
"pivot_브랜드코드_6459",
"pivot_중분류_지갑_벨트",
"pivot_브랜드코드_6947",
"pivot_중분류_행사구두(5F)",
"지점코드별_구매횟수_std",
"구매건수",
"pivot_브랜드코드_5370",
"pivot_중분류_손수건",
"주구매요일",
"평균구매요일",
"pivot_브랜드코드_5930",
"pivot_중분류_TOP디자이너",
"년도별_구매비율_std",
"pivot_브랜드코드_6411",
"주구매_대분류_명품잡화",
"주구매_대분류_명품",
"대분류별_구매횟수_std",
"중분류별_구매횟수_std",
"최소환불금액",
"평균환불금액",
"요일별_구매비율_kurt",
"월별평균구매건수",
"일별평균구매건수",
"방문일수_대비_구매건수",
"구매간격_최소값_x",
"구매간격_최소값_y",
"분기별_구매비율_skew",
"계절별_구매비율_skew",
"백화점이용기간",
"pivot_브랜드코드_5969",
"pivot_중분류_유아발육기",
"18시기준_구매비율_std",
"pivot_브랜드코드_5749",
"pivot_중분류_화장품",
"총방문일수",
"최소구매금액",
"평일_18시이전_구매비율",
"18시이전_구매비율",
"pivot_브랜드코드_6977",
"pivot_브랜드코드_6528",
"연말연시_구매비율",
"12월_구매비율",
"중가제품_구매비율",
"방문지점수",
"평일_구매비율"
  ])


In [72]:
test_ft_tmp = test_ft.drop(columns=["pivot_브랜드코드_5171",
"pivot_중분류_색조화장품",
"첫구매년도",
"선호브랜드코드",
"마지막구매년도",
"브랜드코드별_구매횟수_kurt",
"선호브랜드코드_구매비율",
"평균구매시간",
"주구매시간대",
"월별_구매비율_skew",
"월별_구매비율_std",
"성수기_구매집중도",
"전체_성수기_구매비율",
"pivot_브랜드코드_5208",
"pivot_중분류_홈데코",
"pivot_브랜드코드_6829",
"pivot_중분류_피혁",
"pivot_브랜드코드_6995",
"pivot_브랜드코드_6803",
"계절_구매변동성",
"계절별_구매비율_std",
"분기별_구매비율_std",
"브랜드코드별_구매횟수_skew",
"중분류별_구매횟수_skew",
"주말_방문_선호도",
"주말_구매비율",
"대분류별_구매횟수_skew",
"요일별_구매비율_std",
"pivot_브랜드코드_5433",
"pivot_중분류_유니섹스캐주얼",
"pivot_브랜드코드_5464",
"pivot_중분류_수입캐주얼",
"pivot_브랜드코드_6735",
"pivot_브랜드코드_6506",
"중분류별_구매횟수_kurt",
"주구매_중분류_화장품",
"요일별_구매비율_skew",
"주구매지점_이용비율",
"오후_구매비율",
"구매간격_평균_x",
"구매간격_평균_y",
"구매간격_중앙값_x",
"구매간격_중앙값_y",
"지점코드별_구매횟수_skew",
"pivot_브랜드코드_5962",
"pivot_중분류_골프(용품)",
"대분류별_구매횟수_kurt",
"pivot_브랜드코드_6377",
"pivot_브랜드코드_6053",
"구매간격_최대값_x",
"구매간격_최대값_y",
"구매주기",
"구매금액_median",
"평균구매금액",
"월별_구매비율_kurt",
"pivot_브랜드코드_6459",
"pivot_중분류_지갑_벨트",
"pivot_브랜드코드_6947",
"pivot_중분류_행사구두(5F)",
"지점코드별_구매횟수_std",
"구매건수",
"pivot_브랜드코드_5370",
"pivot_중분류_손수건",
"주구매요일",
"평균구매요일",
"pivot_브랜드코드_5930",
"pivot_중분류_TOP디자이너",
"년도별_구매비율_std",
"pivot_브랜드코드_6411",
"주구매_대분류_명품잡화",
"주구매_대분류_명품",
"대분류별_구매횟수_std",
"중분류별_구매횟수_std",
"최소환불금액",
"평균환불금액",
"요일별_구매비율_kurt",
"월별평균구매건수",
"일별평균구매건수",
"방문일수_대비_구매건수",
"구매간격_최소값_x",
"구매간격_최소값_y",
"분기별_구매비율_skew",
"계절별_구매비율_skew",
"백화점이용기간",
"pivot_브랜드코드_5969",
"pivot_중분류_유아발육기",
"18시기준_구매비율_std",
"pivot_브랜드코드_5749",
"pivot_중분류_화장품",
"총방문일수",
"최소구매금액",
"평일_18시이전_구매비율",
"18시이전_구매비율",
"pivot_브랜드코드_6977",
"pivot_브랜드코드_6528",
"연말연시_구매비율",
"12월_구매비율",
"중가제품_구매비율",
"방문지점수",
"평일_구매비율"
 ])


In [ ]:
from sklearn.feature_selection import mutual_info_classif

# 상호 정보 계산
mutual_info = mutual_info_classif(train_ft, target)
print(mutual_info)

In [ ]:
# 피처 중요도 출력
feature_importance = model.feature_importances_
print(feature_importance)

In [ ]:
#교차 검증 (Cross-validation)과 피처 선택
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
score = cross_val_score(model, train_ft, target, cv=5)
print(score.mean())

# 정답 데이터

In [73]:
target = train_target["target"]
target

0        1.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
14935    0.0
14936    0.0
14937    0.0
14938    1.0
14939    1.0
Name: target, Length: 14940, dtype: float64

In [ ]:
%pip install xgboost

In [ ]:
%pip install catboost

In [ ]:
from xgboost import XGBClassifier

model =  XGBClassifier(random_state=SEED)
model.fit(train_ft_tmp, target)


In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=SEED)
model.fit(train_ft_tmp, target)

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(random_state=SEED)
model.fit(train_ft_tmptmp, target)

In [ ]:
train_ft = train_ft.drop(["대분류별_구매횟수_std", "대분류별_구매횟수_skew", "대분류별_구매횟수_kurt"], axis=1)

In [ ]:
test_ft = test_ft.drop(["대분류별_구매횟수_std", "대분류별_구매횟수_skew", "대분류별_구매횟수_kurt"], axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression

# 설정된 SEED를 사용해 모델 초기화
model = LogisticRegression(random_state=SEED)
model.fit(train_ft, target)

In [ ]:
%pip install --upgrade numpy

In [ ]:
train_ft.shape, target.shape

In [ ]:
%pip install catboost==1.0.6

In [ ]:
%pip install numpy==1.22

In [ ]:
from catboost import CatBoostClassifier

# CatBoostClassifier 모델 생성 및 학습
model = CatBoostClassifier(random_state=SEED, verbose=-1)  # verbose=0으로 설정하여 출력 억제
model.fit(train_ft_tmp, target)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 모델 생성 및 학습
model = RandomForestClassifier(random_state=SEED)
model.fit(train_ft, target)

# 하이퍼파라미너 튜닝

In [ ]:
train_ft_tmp.shape

In [ ]:
train_ft_tmp32 = train_ft_tmp.astype('float32')

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from tqdm import tqdm
import time

# 100번 반복하면서 진행 상황을 표시
for i in tqdm(range(100)):
    time.sleep(0.1)  # 각 아이템 처리마다 0.1초 지연


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [-1, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 63],
    'min_child_samples': [20, 50]
}

#param_grid = {
#    'max_iter': [100, 200],           # 최대 반복 횟수
#    'learning_rate': [0.01, 0.1],     # 학습률
#    'max_depth': [3, 5, 7],           # 트리의 최대 깊이
#    'min_samples_leaf': [20, 50],     # 리프 노드의 최소 샘플 수
#    'max_bins': [255, 512]            # 데이터 분할을 위한 최대 구간 수
#}

grid_search = GridSearchCV(
    LGBMClassifier(random_state=SEED),
    param_grid,
    cv=cv,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(train_ft_tmp, target)

print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

In [ ]:
from tqdm import tqdm
import time

# 100번 반복하면서 진행 상황을 표시
for i in tqdm(range(100)):
    time.sleep(0.1)  # 각 아이템 처리마다 0.1초 지연


# 최적 파라미터 확인
print("Best parameters:", grid_search.best_params_)
print("기존 CV 점수:", 0.722)
print("향상된 CV 점수:", grid_search.best_score_)
print("성능 향상도:", round((grid_search.best_score_ - 0.722) * 100, 2), "%")

# 최적 파라미터로 모델 학습
best_model = LGBMClassifier(**grid_search.best_params_, random_state=SEED)
best_model.fit(train_ft_tmp, target)

# 새로운 예측
pred = best_model.predict(test_ft_tmp)
submit["target"] = pred
submit.to_csv(f"{DATA_PATH}submit_tuned_tmp_HGBC.csv", index=False)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score

# 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(train_ft_tmp, target, test_size=0.2, random_state=42)

# 모델 생성
model = LGBMClassifier(random_state=42)

# 하이퍼파라미터 튜닝을 위한 그리드 서치
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200],
    'num_leaves': [31, 50],
    'max_depth': [3, 5, 7],
    'min_child_samples': [10, 20]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid_search.fit(X_train, y_train)

# 최적의 파라미터와 성능 출력
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best F1 Score: {grid_search.best_score_:.4f}")

# 검증 데이터로 예측
y_pred = grid_search.predict(test_ft_tmp)

# 모델 평가
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1 Score: {f1:.4f}")

# 피처 삭제하기

In [ ]:
from lightgbm import LGBMClassifier

# 모델 학습
model = LGBMClassifier(random_state=42, verbose=0)
model.fit(train_ft, target)

# 피처 중요도 추출 (booster_ 객체의 get_score 사용)
feature_importance = model.booster_.get_score(importance_type='split')  # or 'gain'
features = train_ft.columns

# 중요도가 낮은 피처 삭제 (예: 중요도 10 이하인 피처 제거)
low_importance_features = [feature for feature, importance in feature_importance.items() if importance < 10]

# 삭제
train_ft_cleaned = train_ft.drop(columns=low_importance_features)

# 결과 출력
print(f"삭제된 피처: {low_importance_features}")
print(f"남은 피처: {train_ft_cleaned.columns}")

In [ ]:
import statsmodels.api as sm
import pandas as pd
from tqdm import tqdm
import time


# 100번 반복하면서 진행 상황을 표시
for i in tqdm(range(100)):
    time.sleep(0.1)  # 각 아이템 처리마다 0.1초 지연

    

# 데이터 예시 (df와 target은 사용자의 데이터로 대체)
X = train_ft  # 타겟 변수 제외
y = target

# 모든 피처를 포함한 모델 초기화
X_with_const = sm.add_constant(X)  # 절편 추가
model = sm.OLS(y, X_with_const).fit()

# p-value 기준으로 후진 제거 수행
while True:
    # 가장 높은 p-value를 가진 변수 식별
    max_p_value = model.pvalues.max()
    if max_p_value > 0.05:  # 유의수준 기준 (예: 0.05)보다 높은 경우
        excluded_feature = model.pvalues.idxmax()
        X_with_const = X_with_const.drop(columns=[excluded_feature])
        model = sm.OLS(y, X_with_const).fit()  # 모델 재학습
    else:
        break  # 모든 p-value가 기준 이하일 경우 종료

print("최종 선택된 피처:", X_with_const.columns)
print("최종 모델 요약:", model.summary())

# 피쳐 중요도

In [ ]:
from lightgbm import LGBMClassifier
import pandas as pd

# 모델 학습
model = LGBMClassifier(random_state=42, verbose=0)
model.fit(train_ft, target)

# Booster 객체를 사용하여 피처 중요도 추출
feature_importance = model.booster_.get_score(importance_type='split')  # 'split' 또는 'gain'
features = train_ft.columns

# 중요도 값으로 DataFrame 생성
feature_importance_df = pd.DataFrame({
    'Feature': list(feature_importance.keys()),
    'Importance': list(feature_importance.values())
})

# 중요도 기준 내림차순으로 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# 결과 출력
print(feature_importance_df)

In [ ]:
# 모델 학습
model = LGBMClassifier(random_state=42, verbose=0)
model.fit(train_ft, target)

# 피처 중요도 추출
feature_importance = model.get_feature_importance(importance_type='split')  # 또는 'gain'
features = train_ft.columns

# 중요도 기준 내림차순으로 정렬
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importance
})

# 중요도 기준 내림차순으로 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# 결과 출력
print(feature_importance_df)

# cv 점수 확인해보기

In [71]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

model = LogisticRegression(random_state=SEED)
scores = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1)
np.mean(scores)

In [ ]:
from lightgbm import LGBMClassifier #np.float64(0.7203572651967882) 0.720125 <-0.9 |  train_ft np.float64(0.7179343270567854)

model = LGBMClassifier(random_state=SEED)
scores = cross_val_score(model, train_ft_tmp, target, cv=cv, scoring="f1_macro", n_jobs=-1)
np.mean(scores)

In [ ]:
from lightgbm import LGBMClassifier #np.float64(0.7203572651967882) 0.720125

model = LGBMClassifier(random_state=SEED)
scores = cross_val_score(model, train_ft_tmptmp, target, cv=cv, scoring="f1_macro", n_jobs=-1)
np.mean(scores)

In [ ]:
#np.float64(0.7203572651967882) 0.720125 #0.9만삭제 

In [ ]:
np.float64(0.7187841640982591)

In [ ]:
from xgboost import XGBClassifier #np.float64(0.704564392307905) #0.9

model = XGBClassifier(random_state=SEED)
scores = cross_val_score(model, train_ft_tmp, target, cv=cv, scoring="f1_macro", n_jobs=-1)
np.mean(scores)

In [ ]:
from xgboost import XGBClassifier #np.float64(0.704564392307905) #0.9      

model2 = XGBClassifier(random_state=SEED)
scores2 = cross_val_score(model2, train_ft_tmp, target, cv=cv, scoring="f1_macro", n_jobs=-1)
np.mean(scores2)

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(random_state=SEED, verbose=0)
scores = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1)
np.mean(scores)

In [ ]:
import numpy as np

# 랜덤포레스트 모델 생성
model = RandomForestClassifier(random_state=SEED)
# 교차 검증 수행
scores = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1)

# 평균 F1 점수 계산 0.66
mean_score = np.mean(scores)
print("Mean F1 Macro Score:", mean_score)

# 모델 학습

In [ ]:
model = XGBClassifier(random_state=SEED)
model.fit(train_ft_tmp, target)

In [ ]:

model = CatBoostClassifier(random_state=SEED, verbose=0)
model.fit(train_ft, target)

In [ ]:
model = LGBMClassifier(random_state=SEED)
model.fit(train_ft_tmp,target)

In [ ]:
# 랜덤포레스트 모델 생성 및 학습
model = RandomForestClassifier(random_state=SEED)
model.fit(train_ft, target)

# 테스트 데이터 예측

In [ ]:
pred = model.predict(test_ft_tmp)
pred

In [ ]:
pred2 = model2.predict(test_ft_tmp)
pred2

In [ ]:
# pred = model.predict_proba(test_ft)[:,1]
# pred

# 평가를 위한 제출 파일 생성
- 예측 결과를 target 컬럼에 넣어 csv 파일로 저장후에 제출한다.

In [ ]:
submit

In [ ]:
submit["target"] = pred
submit

In [ ]:
submit["target"] = LX
submit

# 앙상블

In [ ]:
LX = (pred *0.5) + (pred2*0.5)

- 예측 결과를 csv 파일로 저장하여 제출

In [ ]:
submit.to_csv(f"{DATA_PATH}submit_LX.csv",index=False)

In [ ]:
train_ft.shape, test_ft.shape

In [ ]:
num_train = len(train_ft_tmp)

X = train_ft_tmp[:num_train]
X_test = test_ft_tmp

y=target.values

# 지니계수

In [ ]:
import numpy as np
def eval_gini(y_true, y_pred):
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0]
    L_mid = np.linspace(1/n_samples, 1, n_samples) 

    pred_order = y_true[y_pred.argsort()]
    L_pred = np.cumsum(pred_order)/np.sum(pred_order)
    G_pred = np.sum (L_mid-L_pred)

    true_order = y_true[y_true.argsort()]
    L_true = np.cumsum(true_order)/np.sum(true_order)
    G_true = np.sum(L_mid - L_true)

    return G_pred/G_true


In [ ]:
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds), True

# K폴드 교차검증

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)
params = {'objective' : 'binary', 
          'learning_rate' : 0.01,
          'force_row_wise' : True,
          'random_state' : 0}

oof_val_preds = np.zeros(X.shape[0])
oof_test_preds= np.zeros(X_test.shape[0] )

# AutoML

In [ ]:
%pip install flaml

In [ ]:
from flaml import AutoML
auto_ml = AutoML()

In [ ]:
SEED = 42

In [ ]:
params = {
    "metric": "roc_auc",
    "task" : "classification",
    "time_budget": 60*3,
    "seed" : SEED,
    "early_stop": True
}

auto_ml.fit(train_ft, target, **params)

In [ ]:
%pip install tpot

In [ ]:
import pandas as pd
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터 로드
#train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv")  # 학습용 거래 데이터
#target = pd.read_csv(f"{DATA_PATH}store_train.csv")  # 학습용 정답 레이블 데이터

# 학습 데이터 준비
#X = train_tr  # 학습용 피처 (train_tr)
#y = target  # 타겟 레이블

# 학습 데이터와 테스트 데이터로 분할 (80% 학습, 20% 테스트)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# AutoML 모델 설정 (TPOT 사용)
tpot = TPOTClassifier(verbosity=2, generations=5, population_size=20, random_state=42)

# 모델 학습
#tpot.fit(train_ftr, target)

# 예측
#y_pred = tpot.predict(X_test)

# 성능 평가 (정확도)
accuracy = accuracy_score(train_tr, target)
print(f"Accuracy: {accuracy}")

# 최적 모델 파이프라인 저장
tpot.export('best_model_pipeline.py')

# 테스트 데이터에 대한 예측 (추가 예시)
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv")  # 테스트 데이터 (추가 예시)
test_pred = tpot.predict(test_tr)

# 제출 양식에 맞게 예측값 저장 (추가 예시)
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv")  # 제출 양식 데이터
submit['target'] = test_pred  # 예측값을 'target' 컬럼에 넣기

# 제출 파일 저장
submit.to_csv('submission.csv', index=False)

In [ ]:
import pandas as pd
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터 로드
#train_ft = pd.read_csv(f"{DATA_PATH}train_common.csv")  # 학습 데이터(피처)
#target = pd.read_csv(f"{DATA_PATH}store_train.csv")  # 학습용 정답 데이터

# 학습 데이터 준비
X = train_ft  # 피처
y = target  # 타겟 레이블

# 학습 데이터와 테스트 데이터로 분할 (80% 학습, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# AutoML 모델 설정 (TPOT 사용)
tpot = TPOTClassifier(verbosity=2, generations=5, population_size=20, random_state=42)

# 모델 학습
tpot.fit(X_train, y_train)

# 예측
y_pred = tpot.predict(X_test)

# 성능 평가 (정확도)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# 최적 모델 파이프라인 저장
tpot.export('best_model_pipeline.py')


In [ ]:
# 예측값을 제출 파일 형식에 맞게 저장 (추가 예시)
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv")  # 제출 양식 데이터
submit['target'] = tpot.predict(X_test)  # 예측값을 'target' 컬럼에 넣기

# 제출 파일 저장
submit.to_csv('submission.csv', index=False)

In [ ]:
num_train = len(train_ft_tmp)

X = train_ft_tmp[:num_train]
X_test = test_ft_tmp

y=target

In [ ]:
y

In [ ]:
X_test.head()

In [ ]:
X.shape, X_test.shape


In [ ]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)
params = {'objective': 'binary',
          'learning_rate': 0.01,
          'force_row_wise' : True,
          'random_state': 0}
oof_val_preds = np.zeros(X.shape[0])
oof_test_preds = np.zeros(X_test.shape[0])

In [ ]:
import lightgbm as lgb

for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)

    X_train, y_train = X.values[train_idx], y.values[train_idx]
    X_valid, y_valid = X.values[valid_idx], y.values[valid_idx]

    dtrain = lgb.Dataset(X_train, y_train)
    dvalid = lgb.Dataset(X_valid, y_valid)

    lgb_model = lgb.train(params=params,
                          train_set=dtrain,
                          num_boost_round=1000,
                          valid_sets=dvalid,
                          feval=gini,
                          #early_stopping_rounds=100,
                          #verbose_eval=100
                          )
    
    oof_test_preds += lgb_model.predict(X_test) / folds.get_n_splits()
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)

    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수: {gini_score}\n')

In [ ]:
print('OOF 검증 데이터 지니계수 :', eval_gini(y, oof_val_preds))

In [ ]:
submit

In [ ]:
#submit["target"] = oof_test_preds
#submit.to_csv('submission_g.csv')

# 앙상블? (피처갯수가 500-600개면 트리들이 있으니까 피처 셀렉션할필요가없음) -> 앙상블모델이아닌 로지스틱들은 해야함... 

In [ ]:
MLP는 튜닝해야함, 히스토그램 그레이디언트(앙상블):히스토그램으로 구간별로 그레이디언트부스팅 이거 성능잘나올것, LDA(모델) 피처셀렉션해야함(리니어), 
브랜드코드TOP100(구매건수 등 활용) 앙상블은 셀렉션하지말고(피처정보 유실 가능, 단일모델은 하는게 좋음 무조건은 아닌데)
튜닝하지말고 기본파라미터에서 CV점수만 모아서 보팅이나 스태킹해보는 것
캐글의 퍼블릭 점수를 의식하면 안됨... 30%.. AUC 점수가 소숫점 3자리에서 차이나는거면 많이 나는것 ..F1스코어 3자리는 프라이빗해서 뒤집힘. 
앙상블을 해줘야함..
히스트gbm, 앙상블 가르쳐줄때 for문 직접 돌리면서 했떤 것 -하나의 모델에서 5개가 나옴 그걸 다시 평균내고, 다른모델들도 평균내고 다시 평균내서 제출을해보는 시도...? 
캣부스트는 튜닝하면 안되고... 다양한 모델에서 CV 0.7이상이면 로지스틱 등 이면 괜찮음

LDA가 잘안나온다.. 

In [ ]:
피처중요도 목요일구매비율 요일별로도 피벗테이블을 만들면 좋을것 같고, 브랜드코드 삭제할지 고민할것 아니면 줄일지 (5-600개) 800개정도로 마지노선 잡아서 

In [ ]:
num_train = len(train_ft_tmp)

X = train_ft_tmp[:num_train]
X_test = test_ft_tmp

y=target

In [ ]:
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# 데이터셋 로드
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 정의
lgbm = LGBMClassifier()
xgb = XGBClassifier()

# Voting Classifier 생성 (Soft Voting)
voting_clf = VotingClassifier(estimators=[
    ('lgbm', lgbm),
    ('xgb', xgb)
], voting='soft')

# 모델 학습
voting_clf.fit(X_train, y_train)

# 예측 및 평가
y_pred = voting_clf.predict(X_test)
print(f'Voting Classifier Accuracy: {accuracy_score(y_test, y_pred)}')

In [ ]:
target.info

In [ ]:
import pandas as pd
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# 모델 정의
xgb_model = XGBClassifier()
lgbm_model = LGBMClassifier()

# Voting 앙상블 생성
voting_clf = VotingClassifier(estimators=[
    ('xgb', xgb_model),
    ('lgbm', lgbm_model)
], voting='soft')

# 모델 학습
voting_clf.fit(train_ft_tmp, target)

# 예측 수행
y_pred = voting_clf.predict(test_ft_tmp)

In [ ]:
# 제출 파일 생성
submission = pd.DataFrame({
    'ID': submit['ID'],  # test_ft에 ID가 포함되어 있다고 가정
    'Prediction': y_pred
})

# CSV 파일로 저장
submission.to_csv('submission_A.csv', index=False)

print("Submission file saved as submission.csv")

In [ ]:
# 제출 파일 생성 (예: ID와 예측값으로 구성된 데이터프레임 생성)
submission = pd.DataFrame({
    'ID': submit['ID'],  # 테스트 데이터에서 ID 컬럼 사용
    'target': y_pred
})

# CSV 파일로 저장
submission.to_csv('submission_LX.csv', index=False)

print("Prediction saved to submission.csv")

# feature importance

In [ ]:
import lightgbm as lgb
import matplotlib.pyplot as plt

# LGBM 모델 학습 (예시)
lgb_model = lgb.LGBMClassifier()
lgb_model.fit(train_ft, target)

# Feature Importance 시각화
lgb.plot_importance(lgb_model, max_num_features=200, importance_type='split', figsize=(10, 30))
plt.show()



# Feature Importance 값 추출
importance_df = pd.DataFrame({
    'Feature': train_ft.columns,  # 피처명
    'Importance': lgb_model.feature_importances_  # 피처 중요도 값
})

# 중요도 값 기준으로 내림차순 정렬
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# 엑셀로 저장
importance_df.to_excel('feature_importance_col_2.xlsx', index=False)

# 결과 확인
print(importance_df.head())  # 상위 몇 개의 피처 확인

# 앙상블(11/12)

In [72]:
import matplotlib
from matplotlib import font_manager

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv

fontpaths = ["/usr/share/fonts/truetype/nanum/"]
font_files = font_manager.findSystemFonts(fontpaths=fontpaths)

for ff in font_files:
    font_manager.fontManager.addfont(ff)

matplotlib.rc('font', family="NanumGothic")
matplotlib.rcParams['axes.unicode_minus'] = False

'sudo'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'sudo'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [73]:
# 보팅

In [74]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
cv = StratifiedKFold(5, shuffle=True, random_state=42)

In [ ]:
%pip install numpy

In [ ]:
%pip show numpy

In [ ]:
%pip install --upgrade numpy

In [ ]:
%pip show numpy

In [ ]:
%pip install Ca

In [ ]:
%pip install numpy==1.23.5

In [75]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
estimators = [
   # ("dt", DecisionTreeClassifier(random_state=42) ),
   # ("hgb", HistGradientBoostingClassifier(max_iter=200, class_weight='balanced', random_state=42) ),
    ("hgb", HistGradientBoostingClassifier(max_iter=200, class_weight={0: 1, 1: 3}, random_state=42) ),
    #("knn", KNeighborsClassifier() ),
    ("lgb", LGBMClassifier() ),
    ("cbc", CatBoostClassifier() ),
    ("rfc", RandomForestClassifier(class_weight={0: 1, 1: 3}, random_state=42)),
    ("lr", LogisticRegression(class_weight={0: 1, 1: 3}, random_state=42))

]




In [77]:
from sklearn.ensemble import VotingClassifier
parmas = {
    "estimators": estimators,
    "voting" : "soft",
    "n_jobs" : -1
}

model = VotingClassifier(**parmas)

In [78]:
from sklearn.model_selection import cross_val_score

# train_ft_tmp와 target을 numpy 배열로 변환
train_ft_X = train_ft_tmp.values  # DataFrame을 numpy 배열로 변환
target_y = target.values        # DataFrame을 numpy 배열로 변환
test_ft_X = test_ft_tmp.values

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

# train_ft_tmp와 target을 numpy 배열로 변환
X = train_ft_tmp.to_numpy()  # 또는 X = train_ft_tmp.values
y = target.to_numpy()        # 또는 y = target.values

In [ ]:
scores = cross_val_score(model, train_ft_X, target_y, cv=cv, scoring="f1_macro", n_jobs=-1)
np.mean(scores)

In [ ]:
#hist, lgb, xgb 0.7218528459950463
#hist, lgb, cbc 0.7246023688333636
#hist, lgb, cbc, rfc, lr 0.7246023688333636
#hist, lgb, cbc, rfc, 0.7210786185297428
#hist, lgb, cbc, rfc, 0.7182613652949599 #스탠다드스케일러 

In [ ]:
model.fit(train_ft_X,target)

In [ ]:
pred = model.predict(test_ft_X)

In [ ]:
submit["target"] = pred
submit

In [ ]:
submit.to_csv(f"{DATA_PATH}submit_HLC.csv",index=False)

In [ ]:
# 하이퍼파라미터까지 튜닝
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

# 기본 모델 리스트
estimators = [
    ("hgb", HistGradientBoostingClassifier(random_state=42)),
    ("lgb", LGBMClassifier(random_state=42)),
    ("cbc", CatBoostClassifier(random_state=42, verbose=0))  # verbose를 0으로 설정
]

# VotingClassifier 설정
voting_model = VotingClassifier(estimators=estimators, voting="soft", n_jobs=-1)

# 각 모델에 대해 하이퍼파라미터 그리드 설정
param_grids = {
    'hgb__max_iter': [100, 200],
    'lgb__num_leaves': [31, 50],
    'cbc__iterations': [500, 1000],
    'cbc__learning_rate': [0.01, 0.05]
}

# GridSearchCV 설정
grid_search = GridSearchCV(estimator=voting_model, param_grid=param_grids, cv=5, scoring="f1_macro", n_jobs=-1, verbose=1)

# 모델 학습
grid_search.fit(train_ft_X, target)

# 최적의 모델 출력
print("Best Hyperparameters:", grid_search.best_params_)

# 최적 모델로 예측
best_model = grid_search.best_estimator_
y_pred = best_model.predict(test_ft_X)

# 모델 평가
f1 = f1_score(target_y, y_pred)
print(f"Validation F1 Score: {f1:.4f}")

In [ ]:
train_ft_X.shape

In [ ]:
# 스태킹

In [ ]:
from sklearn.ensemble import StackingClassifier
params = {
    "estimators": estimators,
    "final_estimator": LGBMClassifier(random_state=42),
    "cv" : cv,
    "n_jobs": -1
}
model = StackingClassifier(**params)

scores = cross_val_score(model, train_ft_X, target, cv=cv,scoring="f1_macro", n_jobs=-1)
scores.mean()

In [ ]:
#hist, lgb, cbc, rfc, 0.7207675673042508 #스탠다드스케일러  

In [ ]:
model.fit(train_ft_X, target)
model.predict(test_ft_X)

In [ ]:
pred = model.predict(test_ft_X)
pred

In [ ]:
submit

In [ ]:
submit["target"] = pred
submit

In [ ]:
submit.to_csv(f"{DATA_PATH}submit__.csv",index=False)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

# 훈련 데이터와 검증 데이터로 분할
X_train, X_val, y_train, y_val = train_test_split(train_ft_tmp, target, test_size=0.2, random_state=42)

# StackingClassifier 모델 설정
params = {
    "estimators": estimators,
    "final_estimator": LogisticRegression(random_state=42),
    "cv": cv,
    "n_jobs": -1
}

model = StackingClassifier(**params)

# 모델 학습
model.fit(X_train, y_train)

# 검증 데이터로 예측
y_pred = model.predict(X_val)

# F1 점수 계산
f1 = f1_score(y_val, y_pred, average='macro')
print(f"F1 Score (macro) on validation data: {f1}")

# 모델 전체 훈련 데이터에 대해 다시 학습 후 테스트 데이터 예측
model.fit(train_ft_tmp, target)
test_predictions = model.predict(test_ft_tmp)

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

# 데이터 분리 (train_ft에서 train-validation split)
X_train, X_val, y_train, y_val = train_test_split(
    train_ft, target, test_size=0.2, random_state=42
)

# 모델 초기화
model = HistGradientBoostingClassifier(random_state=42)

# 모델 학습
model.fit(X_train, y_train)

# 검증 데이터 예측
y_val_pred = model.predict(X_val)

# 성능 평가 (Validation Set)
accuracy = accuracy_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1 Score: {f1:.4f}")

# 테스트 데이터 예측
test_predictions = model.predict(test_ft)

# 테스트 결과 출력
print("Test Predictions:")
print(test_predictions)

In [ ]:
params = {
    "random_state": 42,
    "n_estimators": 200,
     "learning_rate": 0.2,
    "n_jobs": -1
}

model = XGBClassifier(**params)
model.fit(train_ft,target)
model.predict(test_ft)

In [ ]:
from lightgbm import LGBMClassifier
params = {
    "random_state": 42,
    "n_estimators": 200,
    "learning_rate": 0.2,
    "n_jobs": -1
}

model = LGBMClassifier(**params)
scores = cross_val_score(model,train_ft_tmp, target, cv=cv, scoring="f1", n_jobs=-1)
scores.mean()

In [ ]:
def f1_lgbm_metric(true, pred):
    pred = (pred >= 0.5).astype(int)
    return "f1-score", f1_score(true,pred), True

In [ ]:
params = {
    "random_state": 42,
    "n_estimators": 1000,
     "learning_rate": 0.2,
    "n_jobs": -1,
    "early_stopping_round": 50,
    # "metric": "auc",
}
scores = []
models = []
for tri, vai in cv.split(train_ft_tmp, target):
    # 학습데이터
    x_train = train_ft_tmp.iloc[tri]
    y_train = target.iloc[tri]

    # 검증 데이터
    x_valid = train_ft_tmp.iloc[vai]
    y_valid = target.iloc[vai]

    model = LGBMClassifier(**params)
    model.fit(x_train,y_train, eval_set = [(x_valid, y_valid)], eval_metric = f1_lgbm_metric)

    models.append(model)

    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred)
    scores.append(score)

In [ ]:
np.mean(scores)

# 히스토그램 그레이디언트(11/13)

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(train_ft_tmp, target, test_size=0.2, random_state=42)

# Histogram-based Gradient Boosting 모델 생성 및 학습
model = HistGradientBoostingClassifier(random_state=42)

# 모델 학습
model.fit(X_train, y_train)

# 검증 데이터 예측
y_pred = model.predict(X_val)

# 모델 평가
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

# 테스트 데이터 예측
test_predictions = model.predict(test_ft_tmp)

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(train_ft_tmp, target, test_size=0.2, random_state=42)

# Histogram-based Gradient Boosting 모델 생성
model = HistGradientBoostingClassifier(random_state=42)

# 교차 검증 점수 계산 (F1 매크로 스코어)
cv_scores = cross_val_score(model, train_ft_tmp, target, cv=5, scoring='f1_macro', n_jobs=-1)

print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean CV F1 Score: {cv_scores.mean():.4f}\n")

# 모델 학습
model.fit(X_train, y_train)

# 검증 데이터 예측
y_pred = model.predict(X_val)

# 모델 평가
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='macro')  # 다중 클래스 대비 매크로 F1 스코어

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

# 테스트 데이터 예측
test_predictions = model.predict(test_ft_tmp)

In [ ]:
submit["target"] = test_predictions
submit

In [ ]:
submit.to_csv(f"{DATA_PATH}submit_HGBC.csv",index=False)

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(train_ft_tmp, target, test_size=0.2, random_state=42)

# Histogram-based Gradient Boosting 모델 생성
model = HistGradientBoostingClassifier(random_state=42)

# 교차 검증 점수 계산 (F1 매크로 스코어)
cv_scores = cross_val_score(model, train_ft_tmp, target, cv=10, scoring='f1_macro', n_jobs=-1)

print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean CV F1 Score: {cv_scores.mean():.4f}\n")

# 모델 학습
model.fit(X_train, y_train)

# 검증 데이터 예측
y_pred = model.predict(X_val)

# 모델 평가
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='macro')  # 다중 클래스 대비 매크로 F1 스코어

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

# 테스트 데이터 예측
test_predictions = model.predict(test_ft_tmp)

# 앙상블+recall 부분 수정

In [75]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In [76]:

estimators = [
   # ("dt", DecisionTreeClassifier(random_state=42) ),
    #("hgb", HistGradientBoostingClassifier(max_iter=200, class_weight='balanced', random_state=42) ),
     ("hgb", HistGradientBoostingClassifier(max_iter=200, class_weight={0: 1, 1: 2}, random_state=42) ),
    #("knn", KNeighborsClassifier() ),
    ("lgb", LGBMClassifier() ),
    ("cbc", CatBoostClassifier() ),
    ("rfc", RandomForestClassifier(class_weight={0: 1 ,1: 2}, random_state=42)),
    #("lr", LogisticRegression(class_weight={0: 1, 1: 2}, random_state=42))

]



from sklearn.ensemble import VotingClassifier
parmas = {
    "estimators": estimators,
    "voting" : "soft",
    "n_jobs" : -1
}

model = VotingClassifier(**parmas)

In [77]:

from sklearn.model_selection import cross_val_score

# train_ft_tmp와 target을 numpy 배열로 변환
train_ft_X = train_ft_tmp.values  # DataFrame을 numpy 배열로 변환
target_y = target.values        # DataFrame을 numpy 배열로 변환
test_ft_X = test_ft_tmp.values

In [161]:

from sklearn.model_selection import cross_val_score

# train_ft_tmp와 target을 numpy 배열로 변환
train_ft_X = train_ft.values  # DataFrame을 numpy 배열로 변환
target_y = target.values        # DataFrame을 numpy 배열로 변환
test_ft_X = test_ft.values

In [85]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
import numpy as np

# SMOTE 오버샘플링 적용 (훈련 데이터 전체에 적용)
smote = SMOTE(sampling_strategy=0.85, random_state=42)
train_ft_X_resampled, target_y_resampled = smote.fit_resample(train_ft_X, target_y)

# 클래스 분포 확인
print(f"Before SMOTE - target_y distribution:\n{pd.Series(target_y).value_counts()}")
print(f"After SMOTE - target_y_resampled distribution:\n{pd.Series(target_y_resampled).value_counts()}")

# 모델 생성
model = VotingClassifier(**parmas)

# 교차 검증 점수 계산 (훈련 데이터 전체로)
cv_scores = cross_val_score(model, train_ft_X_resampled, target_y_resampled, cv=5, scoring='f1_macro', n_jobs=-1)
print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean CV F1 Score: {cv_scores.mean():.4f}\n")

# 모델 학습
model.fit(train_ft_X_resampled, target_y_resampled)

# 훈련 데이터 자체 평가 (검증 데이터 없이 전체 데이터 사용)
y_pred = model.predict(train_ft_X_resampled)

accuracy = accuracy_score(target_y_resampled, y_pred)
f1 = f1_score(target_y_resampled, y_pred, average='macro')

print(f"Train Accuracy: {accuracy:.4f}")
print(f"Train F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(target_y_resampled, y_pred))

# 테스트 데이터 예측
test_predictions = model.predict(test_ft_X)

Before SMOTE - target_y distribution:
0.0    9066
1.0    5874
Name: count, dtype: int64
After SMOTE - target_y_resampled distribution:
0.0    9066
1.0    7706
Name: count, dtype: int64


c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
3 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\_utils.py", line 72, in __call__
    return self.func(**kwargs)
  File "c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\parallel.py", line 598, in __call__
    return [func(*args, **kwargs)
  File "c:\Users\jieun\AppData\Local\Programs\Python\Python310\lib\site-packages\

Cross-Validation F1 Scores: [0.73261151        nan        nan        nan 0.82672396]
Mean CV F1 Score: nan

Train Accuracy: 0.9456
Train F1 Score: 0.9453

Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.94      0.95      9066
         1.0       0.93      0.95      0.94      7706

    accuracy                           0.95     16772
   macro avg       0.94      0.95      0.95     16772
weighted avg       0.95      0.95      0.95     16772



In [86]:
test_probabilities = model.predict_proba(test_ft_X)

In [87]:
test_probabilities

array([[0.87604501, 0.12395499],
       [0.54042862, 0.45957138],
       [0.41125041, 0.58874959],
       ...,
       [0.86645356, 0.13354644],
       [0.69887977, 0.30112023],
       [0.96139549, 0.03860451]])

In [91]:

# ID와 예측 확률을 결합
submission = pd.DataFrame({
    'ID': submit['ID'],  # 테스트 데이터의 ID
    'target': test_probabilities  # 클래스 1에 대한 확률값
})

In [92]:
submission.to_csv('submit_이지은.csv', index=False)

print("제출 파일이 'submission.csv'로 저장되었습니다.")

제출 파일이 'submission.csv'로 저장되었습니다.


In [ ]:
test_probabilities = model.predict_proba(test_ft_X)[:, 1]

In [ ]:
# 예측 확률 확인
print("Test probabilities for class 1:")
print(test_probabilities)

# 결과를 DataFrame으로 저장 (ID가 있는 경우)
submission = pd.DataFrame({
    'ID': test_ids,  # test 데이터의 ID 컬럼
    'Prediction': test_probabilities
})

# CSV 파일로 저장
submission.to_csv('submission.csv', index=False)
print("Submission file saved as 'submission.csv'")

In [83]:
# 테스트 데이터에 대한 클래스 1의 확률 추출
test_probabilities = model.predict_proba(test_ft_X)[:, 1]

NotFittedError: This VotingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [82]:
# 테스트 데이터에 대한 클래스 1의 확률 추출
test_probabilities = model.predict_proba(test_ft_X)[:, 1]

# 결과를 DataFrame으로 저장 (ID가 있는 경우 함께 저장)
submission = pd.DataFrame({
    'ID': submit  # test 데이터의 ID 컬럼
    'Prediction': test_probabilities
})

# CSV 파일로 저장
submission.to_csv('submit_이지은.csv', index=False)

print("Submission file saved as 'submission.csv'")

SyntaxError: invalid syntax (592273232.py, line 7)

In [81]:
submit["target"] = test_predictions
submit
submit.to_csv(f"{DATA_PATH}submit_1114_.csv",index=False)

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np

# SMOTE 오버샘플링 적용
smote = SMOTE(sampling_strategy=0.85, random_state=42)
train_ft_X_resampled, target_y_resampled = smote.fit_resample(train_ft_X, target_y)

# 클래스 분포 확인
print(f"Before SMOTE - y_train distribution:\n{pd.Series(target_y).value_counts()}")
print(f"After SMOTE - y_train_resampled distribution:\n{pd.Series(target_y_resampled).value_counts()}")

# 모델 생성
model = VotingClassifier(**parmas)

# 교차 검증 점수 계산 (F1 매크로 스코어)
cv_scores = cross_val_score(model, train_ft_X_resampled, target_y_resampled, cv=5, scoring='f1_macro', n_jobs=-1)
print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean CV F1 Score: {cv_scores.mean():.4f}\n")

# 모델 학습
model.fit(train_ft_X_resampled, target_y_resampled)




# 테스트 데이터 예측
test_predictions = model.predict(test_ft_X)

# 예측 결과 제출 (필요시)
# submission = pd.DataFrame({'ID': test_ids, 'Prediction': test_predictions})
# submission.to_csv('submission.csv', index=False)

Before SMOTE - y_train distribution:
0.0    9066
1.0    5874
Name: count, dtype: int64
After SMOTE - y_train_resampled distribution:
0.0    9066
1.0    7706
Name: count, dtype: int64
Cross-Validation F1 Scores: [0.73049702 0.73230336 0.73676409 0.76631376 0.80636815]
Mean CV F1 Score: 0.7544



In [ ]:
submit["target"] = test_predictions
submit
submit.to_csv(f"{DATA_PATH}submit_1114.csv",index=False)

In [162]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
import numpy as np

# SMOTE 오버샘플링 적용 (훈련 데이터 전체에 적용)
smote = SMOTE(sampling_strategy=0.85, random_state=42)
train_ft_X_resampled, target_y_resampled = smote.fit_resample(train_ft_X, target_y)

# 클래스 분포 확인
print(f"Before SMOTE - target_y distribution:\n{pd.Series(target_y).value_counts()}")
print(f"After SMOTE - target_y_resampled distribution:\n{pd.Series(target_y_resampled).value_counts()}")

# 모델 생성
model = VotingClassifier(**parmas)

# 교차 검증 점수 계산 (훈련 데이터 전체로)
cv_scores = cross_val_score(model, train_ft_X_resampled, target_y_resampled, cv=5, scoring='f1_macro', n_jobs=-1)
print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean CV F1 Score: {cv_scores.mean():.4f}\n")

# 모델 학습
model.fit(train_ft_X_resampled, target_y_resampled)

# 훈련 데이터 자체 평가 (검증 데이터 없이 전체 데이터 사용)
y_pred = model.predict(train_ft_X_resampled)
accuracy = accuracy_score(target_y_resampled, y_pred)
f1 = f1_score(target_y_resampled, y_pred, average='macro')

print(f"Train Accuracy: {accuracy:.4f}")
print(f"Train F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(target_y_resampled, y_pred))

# 테스트 데이터 예측
test_predictions = model.predict(test_ft_X)

Before SMOTE - target_y distribution:
0.0    9066
1.0    5874
Name: count, dtype: int64
After SMOTE - target_y_resampled distribution:
0.0    9066
1.0    7706
Name: count, dtype: int64
Cross-Validation F1 Scores: [0.72172737 0.73484004 0.73778537 0.76062653 0.82255885]
Mean CV F1 Score: 0.7555

Train Accuracy: 0.9468
Train F1 Score: 0.9465

Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.95      0.95      9066
         1.0       0.94      0.95      0.94      7706

    accuracy                           0.95     16772
   macro avg       0.95      0.95      0.95     16772
weighted avg       0.95      0.95      0.95     16772



In [104]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
import numpy as np

# SMOTE 오버샘플링 적용 (훈련 데이터 전체에 적용)
smote = SMOTE(sampling_strategy=0.85, random_state=42)
train_ft_X_resampled, target_y_resampled = smote.fit_resample(train_ft_X, target_y)

# 클래스 분포 확인
print(f"Before SMOTE - target_y distribution:\n{pd.Series(target_y).value_counts()}")
print(f"After SMOTE - target_y_resampled distribution:\n{pd.Series(target_y_resampled).value_counts()}")

# 모델 생성
model = VotingClassifier(**parmas)

# 교차 검증 점수 계산 (훈련 데이터 전체로)
cv_scores = cross_val_score(model, train_ft_X_resampled, target_y_resampled, cv=5, scoring='f1_macro', n_jobs=-1)
print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean CV F1 Score: {cv_scores.mean():.4f}\n")

# 모델 학습
model.fit(train_ft_X_resampled, target_y_resampled)

# 훈련 데이터 자체 평가 (검증 데이터 없이 전체 데이터 사용)
y_pred = model.predict(train_ft_X_resampled)
accuracy = accuracy_score(target_y_resampled, y_pred)
f1 = f1_score(target_y_resampled, y_pred, average='macro')

print(f"Train Accuracy: {accuracy:.4f}")
print(f"Train F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(target_y_resampled, y_pred))

# 테스트 데이터 예측
test_predictions = model.predict(test_ft_X)

Before SMOTE - target_y distribution:
0.0    9066
1.0    5874
Name: count, dtype: int64
After SMOTE - target_y_resampled distribution:
0.0    9066
1.0    7706
Name: count, dtype: int64
Cross-Validation F1 Scores: [0.73261151 0.73143036 0.74295022 0.77001313 0.82672396]
Mean CV F1 Score: 0.7607

Train Accuracy: 0.9456
Train F1 Score: 0.9453

Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.94      0.95      9066
         1.0       0.93      0.95      0.94      7706

    accuracy                           0.95     16772
   macro avg       0.94      0.95      0.95     16772
weighted avg       0.95      0.95      0.95     16772



In [169]:
y_pred

array([1., 1., 0., ..., 1., 1., 1.])

In [108]:
test_predictions

array([0., 0., 1., ..., 0., 0., 0.])

In [107]:
submit

ID  target
0          test_0     0.5
1          test_1     0.5
2          test_2     0.5
3          test_3     0.5
4          test_4     0.5
...           ...     ...
12220  test_12220     0.5
12221  test_12221     0.5
12222  test_12222     0.5
12223  test_12223     0.5
12224  test_12224     0.5

[12225 rows x 2 columns]

In [110]:
submit["target"] = test_predictions

In [111]:
submit

ID  target
0          test_0     0.0
1          test_1     0.0
2          test_2     1.0
3          test_3     1.0
4          test_4     1.0
...           ...     ...
12220  test_12220     1.0
12221  test_12221     0.0
12222  test_12222     0.0
12223  test_12223     0.0
12224  test_12224     0.0

[12225 rows x 2 columns]

In [112]:

submit.to_csv(f"{DATA_PATH}submit_1114_2.csv",index=False)

In [96]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
import numpy as np

# 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(train_ft_X, target_y, test_size=0.2, random_state=42)

# SMOTE 오버샘플링 적용
smote = SMOTE(sampling_strategy=0.9, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 클래스 분포 확인 (Pandas Series로 변환)
print(f"Before SMOTE - y_train distribution:\n{pd.Series(y_train).value_counts()}")
print(f"After SMOTE - y_train_resampled distribution:\n{pd.Series(y_train_resampled).value_counts()}")

# 또는 numpy를 사용한 확인
unique, counts = np.unique(y_train, return_counts=True)
print(f"Before SMOTE - y_train distribution: {dict(zip(unique, counts))}")

unique_resampled, counts_resampled = np.unique(y_train_resampled, return_counts=True)
print(f"After SMOTE - y_train_resampled distribution: {dict(zip(unique_resampled, counts_resampled))}")

# 모델 생성 및 학습
model = VotingClassifier(**parmas)

cv_scores = cross_val_score(model, train_ft_X, target_y, cv=5, scoring='f1_macro', n_jobs=-1)
print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean CV F1 Score: {cv_scores.mean():.4f}\n")

model.fit(X_train_resampled, y_train_resampled)

# 검증 데이터 평가
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='macro')

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

# 테스트 데이터 예측
test_predictions = model.predict(test_ft_X)

Before SMOTE - y_train distribution:
0.0    7265
1.0    4687
Name: count, dtype: int64
After SMOTE - y_train_resampled distribution:
0.0    7265
1.0    6538
Name: count, dtype: int64
Before SMOTE - y_train distribution: {0.0: 7265, 1.0: 4687}
After SMOTE - y_train_resampled distribution: {0.0: 7265, 1.0: 6538}
Cross-Validation F1 Scores: [0.72300877 0.71017994 0.72505072 0.73949678 0.7223474 ]
Mean CV F1 Score: 0.7240

Validation Accuracy: 0.7339
Validation F1 Score: 0.7274

Classification Report:
              precision    recall  f1-score   support

         0.0       0.81      0.74      0.77      1801
         1.0       0.65      0.73      0.69      1187

    accuracy                           0.73      2988
   macro avg       0.73      0.73      0.73      2988
weighted avg       0.74      0.73      0.74      2988



In [ ]:
estimators = [
   # ("dt", DecisionTreeClassifier(random_state=42) ),
    ("hgb", HistGradientBoostingClassifier(max_iter=200, class_weight='balanced', random_state=42) ),
    #("hgb", HistGradientBoostingClassifier(max_iter=200, class_weight={0: 1, 1: 2}, random_state=42) ),
    #("knn", KNeighborsClassifier() ),
    ("lgb", LGBMClassifier() ),
    #("cbc", CatBoostClassifier() ),
    ("rfc", RandomForestClassifier()),
    ("lr", LogisticRegression(class_weight='balanced'))

]

In [79]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(train_ft_tmp, target, test_size=0.2, random_state=42)

# SMOTE 오버샘플링 적용
smote = SMOTE(sampling_strategy=1.0, random_state=42)  # 1.0은 양성 클래스와 음성 클래스의 비율을 같게 만듬
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"Before SMOTE - X_train shape: {X_train.shape}, y_train distribution: {y_train.value_counts()}")
print(f"After SMOTE - X_train_resampled shape: {X_train_resampled.shape}, y_train_resampled distribution: {y_train_resampled.value_counts()}")

# Histogram-based Gradient Boosting 모델 생성
model = VotingClassifier(**parmas)

# 교차 검증 점수 계산 (F1 매크로 스코어)
cv_scores = cross_val_score(model, train_ft_tmp, target, cv=5, scoring='f1_macro', n_jobs=-1)

print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean CV F1 Score: {cv_scores.mean():.4f}\n")

# 모델 학습
model.fit(X_train_resampled, y_train_resampled)

# 검증 데이터 예측
y_pred = model.predict(X_val)

# 모델 평가
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='macro')  # 다중 클래스 대비 매크로 F1 스코어

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

# 테스트 데이터 예측
test_predictions = model.predict(test_ft_tmp)


# 테스트 데이터에 대한 예측 결과를 제출하는 코드 등 추가할 수 있음
# 예: submission = pd.DataFrame({'ID': test_ids, 'Prediction': test_predictions})

Before SMOTE - X_train shape: (11952, 2305), y_train distribution: target
0.0    7265
1.0    4687
Name: count, dtype: int64
After SMOTE - X_train_resampled shape: (14530, 2305), y_train_resampled distribution: target
0.0    7265
1.0    7265
Name: count, dtype: int64


KeyboardInterrupt: 

In [82]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
import numpy as np

# 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(train_ft_X, target_y, test_size=0.2, random_state=42)

# SMOTE 오버샘플링 적용
smote = SMOTE(sampling_strategy=0.7, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 클래스 분포 확인 (Pandas Series로 변환)
print(f"Before SMOTE - y_train distribution:\n{pd.Series(y_train).value_counts()}")
print(f"After SMOTE - y_train_resampled distribution:\n{pd.Series(y_train_resampled).value_counts()}")

# 또는 numpy를 사용한 확인
unique, counts = np.unique(y_train, return_counts=True)
print(f"Before SMOTE - y_train distribution: {dict(zip(unique, counts))}")

unique_resampled, counts_resampled = np.unique(y_train_resampled, return_counts=True)
print(f"After SMOTE - y_train_resampled distribution: {dict(zip(unique_resampled, counts_resampled))}")

# 모델 생성 및 학습
model = VotingClassifier(**parmas)

cv_scores = cross_val_score(model, train_ft_X, target_y, cv=5, scoring='f1_macro', n_jobs=-1)
print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean CV F1 Score: {cv_scores.mean():.4f}\n")

model.fit(X_train_resampled, y_train_resampled)

# 검증 데이터 평가
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='macro')

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

# 테스트 데이터 예측
test_predictions = model.predict(test_ft_X)

Before SMOTE - y_train distribution:
0.0    7265
1.0    4687
Name: count, dtype: int64
After SMOTE - y_train_resampled distribution:
0.0    7265
1.0    5085
Name: count, dtype: int64
Before SMOTE - y_train distribution: {0.0: 7265, 1.0: 4687}
After SMOTE - y_train_resampled distribution: {0.0: 7265, 1.0: 5085}
Cross-Validation F1 Scores: [0.72300877 0.71017994 0.72505072 0.73949678 0.7223474 ]
Mean CV F1 Score: 0.7240

Validation Accuracy: 0.7470
Validation F1 Score: 0.7366

Classification Report:
              precision    recall  f1-score   support

         0.0       0.79      0.78      0.79      1801
         1.0       0.68      0.69      0.68      1187

    accuracy                           0.75      2988
   macro avg       0.74      0.74      0.74      2988
weighted avg       0.75      0.75      0.75      2988



In [84]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
import numpy as np

# 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(train_ft_X, target_y, test_size=0.2, random_state=42)

# SMOTE 오버샘플링 적용
smote = SMOTE(sampling_strategy=0.9, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# 클래스 분포 확인 (Pandas Series로 변환)
print(f"Before SMOTE - y_train distribution:\n{pd.Series(y_train).value_counts()}")
print(f"After SMOTE - y_train_resampled distribution:\n{pd.Series(y_train_resampled).value_counts()}")

# 또는 numpy를 사용한 확인
unique, counts = np.unique(y_train, return_counts=True)
print(f"Before SMOTE - y_train distribution: {dict(zip(unique, counts))}")

unique_resampled, counts_resampled = np.unique(y_train_resampled, return_counts=True)
print(f"After SMOTE - y_train_resampled distribution: {dict(zip(unique_resampled, counts_resampled))}")

# 모델 생성 및 학습
model = VotingClassifier(**parmas)

cv_scores = cross_val_score(model, train_ft_X, target_y, cv=5, scoring='f1_macro', n_jobs=-1)
print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean CV F1 Score: {cv_scores.mean():.4f}\n")

model.fit(X_train_resampled, y_train_resampled)

# 검증 데이터 평가
y_pred= model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='macro')

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

# 테스트 데이터 예측
test_predictions = model.predict(test_ft_X)

Before SMOTE - y_train distribution:
0.0    7265
1.0    4687
Name: count, dtype: int64
After SMOTE - y_train_resampled distribution:
0.0    7265
1.0    6538
Name: count, dtype: int64
Before SMOTE - y_train distribution: {0.0: 7265, 1.0: 4687}
After SMOTE - y_train_resampled distribution: {0.0: 7265, 1.0: 6538}
Cross-Validation F1 Scores: [0.72300877 0.71017994 0.72505072 0.73949678 0.7223474 ]
Mean CV F1 Score: 0.7240

Validation Accuracy: 0.7339
Validation F1 Score: 0.7274

Classification Report:
              precision    recall  f1-score   support

         0.0       0.81      0.74      0.77      1801
         1.0       0.65      0.73      0.69      1187

    accuracy                           0.73      2988
   macro avg       0.73      0.73      0.73      2988
weighted avg       0.74      0.73      0.74      2988



In [86]:
y_pred

array([0., 1., 1., ..., 1., 0., 1.])

In [92]:
test_predictions

array([0., 0., 1., ..., 0., 0., 0.])

In [ ]:
estimators = [
   # ("dt", DecisionTreeClassifier(random_state=42) ),
    #("hgb", HistGradientBoostingClassifier(max_iter=200, class_weight='balanced', random_state=42) ),
     ("hgb", HistGradientBoostingClassifier(max_iter=200, class_weight={0: 1, 1: 2}, random_state=42) ),
    #("knn", KNeighborsClassifier() ),
    ("lgb", LGBMClassifier() ),
    #("cbc", CatBoostClassifier() ),
    ("rfc", RandomForestClassifier(class_weight={0: 1, 1: 2})),
    ("lr", LogisticRegression(class_weight={0: 1, 1: 2}))

]

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(train_ft_tmp, target, test_size=0.2, random_state=42)

# SMOTE 오버샘플링 적용
smote = SMOTE(sampling_strategy=0.8, random_state=42)  # 1.0은 양성 클래스와 음성 클래스의 비율을 같게 만듬
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"Before SMOTE - X_train shape: {X_train.shape}, y_train distribution: {y_train.value_counts()}")
print(f"After SMOTE - X_train_resampled shape: {X_train_resampled.shape}, y_train_resampled distribution: {y_train_resampled.value_counts()}")

# Histogram-based Gradient Boosting 모델 생성
model = VotingClassifier(**parmas)

# 교차 검증 점수 계산 (F1 매크로 스코어)
cv_scores = cross_val_score(model, train_ft_tmp, target, cv=5, scoring='f1_macro', n_jobs=-1)

print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean CV F1 Score: {cv_scores.mean():.4f}\n")

# 모델 학습
model.fit(X_train_resampled, y_train_resampled)

# 검증 데이터 예측
y_pred = model.predict(X_val)

# 모델 평가
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='macro')  # 다중 클래스 대비 매크로 F1 스코어

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

# 테스트 데이터 예측
test_predictions = model.predict(test_ft_tmp)


# 테스트 데이터에 대한 예측 결과를 제출하는 코드 등 추가할 수 있음
# 예: submission = pd.DataFrame({'ID': test_ids, 'Prediction': test_predictions})

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(train_ft_tmp, target, test_size=0.2, random_state=42)

# SMOTE 오버샘플링 적용
smote = SMOTE(sampling_strategy=0.7, random_state=42)  # 1.0은 양성 클래스와 음성 클래스의 비율을 같게 만듬
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"Before SMOTE - X_train shape: {X_train.shape}, y_train distribution: {y_train.value_counts()}")
print(f"After SMOTE - X_train_resampled shape: {X_train_resampled.shape}, y_train_resampled distribution: {y_train_resampled.value_counts()}")

# Histogram-based Gradient Boosting 모델 생성
model = VotingClassifier(**parmas)

# 교차 검증 점수 계산 (F1 매크로 스코어)
cv_scores = cross_val_score(model, train_ft_tmp, target, cv=5, scoring='f1_macro', n_jobs=-1)

print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean CV F1 Score: {cv_scores.mean():.4f}\n")

# 모델 학습
model.fit(X_train_resampled, y_train_resampled)

# 검증 데이터 예측
y_pred = model.predict(X_val)

# 모델 평가
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='macro')  # 다중 클래스 대비 매크로 F1 스코어

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

# 테스트 데이터 예측
test_predictions = model.predict(test_ft_tmp)


# 테스트 데이터에 대한 예측 결과를 제출하는 코드 등 추가할 수 있음
# 예: submission = pd.DataFrame({'ID': test_ids, 'Prediction': test_predictions})

In [ ]:
submit["target"] = test_predictions
submit



In [ ]:
submit.to_csv(f"{DATA_PATH}submit_1113_2.csv",index=False)

# 오버 샘플링

In [ ]:
%pip install imbalanced-learn

In [ ]:
%pip uninstall numpy
%pip install numpy

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 원본 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(train_ft_tmp, target, test_size=0.2, random_state=42)

# SMOTE 오버샘플링 적용
smote = SMOTE(sampling_strategy=1.0, random_state=42)  # 1.0은 양성 클래스와 음성 클래스의 비율을 같게 만듬
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(f"Before SMOTE - X_train shape: {X_train.shape}, y_train distribution: {y_train.value_counts()}")
print(f"After SMOTE - X_train_resampled shape: {X_train_resampled.shape}, y_train_resampled distribution: {y_train_resampled.value_counts()}")

# 모델 학습
model.fit(X_train_resampled, y_train_resampled)

# 검증 데이터 예측
y_pred = model.predict(X_val)

# 모델 평가
accuracy = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='macro')  # 다중 클래스 대비 매크로 F1 스코어

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

# 테스트 데이터 예측
test_predictions = model.predict(test_ft_tmp)
